# Manual Postprocessing - Datos Enriquecidos

Este notebook procesa los datos enriquecidos para:
1. Agrupar entidades por tipo (ciudades, itinerarios, etc.)
2. Normalizar nombres de entidades
3. Combinar entidades similares

In [56]:
import json
import pandas as pd
from collections import defaultdict
from typing import Dict, List, Set
import re

## 1. Cargar Datos Enriquecidos

In [57]:
# Cargar datos enriquecidos
input_file = "../../data/scraper_enrichment/enriched_data.jsonl"

data = []
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.strip()))

print(f"Datos cargados: {len(data)} registros")
print(f"Columnas disponibles: {list(data[0].keys()) if data else 'No hay datos'}")

Datos cargados: 32 registros
Columnas disponibles: ['url', 'source_name', 'scraped_at', 'entities']


## 2. Extraer y Agrupar Entidades

In [58]:
import json
from collections import defaultdict
from pathlib import Path

# Ruta al archivo de entrada
input_path = Path('../../data/scraper_enrichment/enriched_data.jsonl')

# Cargar todas las entidades
all_entities = []
with open(input_path, 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            data = json.loads(line)
            all_entities.extend(data.get('entities', []))

print(f'Total de entidades extraídas: {len(all_entities)}')

# Agrupar entidades por tipo
entidades_por_tipo = defaultdict(list)
for entidad in all_entities:
    tipo = entidad.get('entity_type', 'desconocido')
    
    # Manejar si tipo es una lista
    if isinstance(tipo, list):
        tipo = tipo[0] if tipo else 'desconocido'
    elif tipo is None:
        tipo = 'desconocido'
    
    entidades_por_tipo[tipo].append(entidad)

# Mostrar entidades agrupadas por tipo
for tipo, entidades in entidades_por_tipo.items():
    print(f'\n=== {tipo.upper()} ({len(entidades)}) ===')
    for e in entidades:
        nombre = e.get('name') or e.get('title') or e.get('advice_text') or '[Sin nombre]'
        desc = e.get('description') or e.get('context') or ''
        print(f'- {nombre}: {desc[:120]}')

Total de entidades extraídas: 1516

=== SITE (805) ===
- Tailandia: Diverse country with attractions for all tastes, known for its hospitality, delicious cuisine, and beautiful landscapes 
- Bangkok: Capital city of Thailand with a mix of modernity and tradition, known for its bustling streets, vibrant markets, and cul
- Bangkok: One of the must-see places in Thailand, offering a mix of Western infrastructure and Eastern traditions. Explore the Gra
- Chiang Mai: Known as the 'Northern Capital' of Thailand, offering a magical environment with nature, Buddhist temples, palaces, and 
- Chiang Mai: Explore the city for at least 2 days, with an option for additional excursions. Consider a tour to Chiang Rai from Chian
- Doi Suthep: One of the most beautiful temples to visit in Thailand.
- Phi Phi Islands: Famous island group in Thailand known for its beaches and clear turquoise waters. Includes Koh Phi Phi Don, Koh Phi Phi 
- Sukhothai Historical Park: Historical park located between Bangko

In [59]:
from difflib import SequenceMatcher
from itertools import combinations
import json
from collections import defaultdict
from pathlib import Path

def similarity(a, b):
    """Calcula la similitud entre dos strings (0-1)"""
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def load_entities_from_jsonl(file_path):
    """Carga las entidades desde el archivo JSONL y las agrupa por tipo"""
    entidades_por_tipo = defaultdict(list)
    
    # Convertir a Path object para manejo profesional de rutas
    file_path = Path(file_path)
    
    if not file_path.exists():
        print(f"❌ Error: El archivo {file_path} no existe")
        return entidades_por_tipo
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data = json.loads(line)
                url = data.get('url', 'URL desconocida')
                source_name = data.get('source_name', 'Fuente desconocida')
                
                # Agregar información de origen a cada entidad
                for entidad in data.get('entities', []):
                    entidad['source_url'] = url
                    entidad['source_name'] = source_name
                    
                    tipo = entidad.get('entity_type', 'desconocido')
                    if isinstance(tipo, list):
                        tipo = tipo[0] if tipo else 'desconocido'
                    elif tipo is None:
                        tipo = 'desconocido'
                    
                    entidades_por_tipo[tipo].append(entidad)
    
    return entidades_por_tipo

def find_similar_entities_with_urls(entidades_por_tipo, threshold=0.7):
    """Encuentra entidades similares de TODOS los tipos y las agrupa, mostrando las URLs de origen"""
    
    # Extraer TODAS las entidades con información de origen
    todas_entidades_con_origen = []
    
    for tipo, entidades in entidades_por_tipo.items():
        for entidad in entidades:
            nombre = entidad.get('name') or entidad.get('title') or entidad.get('advice_text') or '[Sin nombre]'
            if nombre and nombre != '[Sin nombre]':
                todas_entidades_con_origen.append({
                    'nombre': nombre,
                    'tipo': tipo,
                    'url': entidad.get('source_url', 'URL desconocida'),
                    'source_name': entidad.get('source_name', 'Fuente desconocida'),
                    'entidad_completa': entidad  # Guardar la entidad completa para más info
                })
    
    if not todas_entidades_con_origen:
        print("No se encontraron entidades")
        return
    
    # Encontrar grupos de entidades similares
    grupos_similares = []
    entidades_usadas = set()
    
    for i, entidad1 in enumerate(todas_entidades_con_origen):
        if entidad1['nombre'] in entidades_usadas:
            continue
            
        grupo = [entidad1]
        entidades_usadas.add(entidad1['nombre'])
        
        # Buscar entidades similares
        for entidad2 in todas_entidades_con_origen[i+1:]:
            if entidad2['nombre'] not in entidades_usadas:
                sim = similarity(entidad1['nombre'], entidad2['nombre'])
                if sim >= threshold and threshold < 0.85:
                    grupo.append(entidad2)
                    entidades_usadas.add(entidad2['nombre'])
        
        if len(grupo) > 1:  # Solo grupos con más de una entidad
            grupos_similares.append(grupo)
    
    # Mostrar resultados ordenados alfabéticamente
    print(f"=== ENTIDADES SIMILARES (similitud >= {threshold}) ===")
    print(f"Total de entidades: {len(todas_entidades_con_origen)}")
    print(f"Grupos de entidades similares: {len(grupos_similares)}")
    
    # Mostrar grupos ordenados alfabéticamente
    for grupo in sorted(grupos_similares, key=lambda x: x[0]['nombre'].lower()):
        print(f"\n📍 {grupo[0]['nombre']} (principal) - Tipo: {grupo[0]['tipo']}")
        print(f"   📍 URL: {grupo[0]['url']}")
        print(f"   📰 Fuente: {grupo[0]['source_name']}")
        
        for entidad in grupo[1:]:
            sim = similarity(grupo[0]['nombre'], entidad['nombre'])
            print(f"   └─ {entidad['nombre']} (similitud: {sim:.2f}) - Tipo: {entidad['tipo']}")
            print(f"      �� URL: {entidad['url']}")
            print(f"      �� Fuente: {entidad['source_name']}")
    
    # Mostrar entidades únicas (sin similares)
    entidades_unicas = [e for e in todas_entidades_con_origen if e['nombre'] not in entidades_usadas or 
                       not any(e['nombre'] in [ent['nombre'] for ent in grupo] for grupo in grupos_similares if len(grupo) > 1)]
    
    if entidades_unicas:
        print(f"\n��️  ENTIDADES ÚNICAS ({len(entidades_unicas)}):")
        for entidad in sorted(entidades_unicas, key=lambda x: x['nombre'].lower()):
            print(f"   • {entidad['nombre']} - Tipo: {entidad['tipo']}")
            print(f"     �� URL: {entidad['url']}")
            print(f"     �� Fuente: {entidad['source_name']}")

# Configurar rutas de forma profesional usando pathlib
current_dir = Path.cwd()
print(f"�� Directorio actual: {current_dir}")

# Construir la ruta al archivo JSONL de forma relativa
file_path = current_dir.parent.parent / "data" / "scraper_enrichment" / "enriched_data.jsonl"
print(f"📄 Archivo objetivo: {file_path}")

# Verificar si el archivo existe
if not file_path.exists():
    print(f"❌ Error: El archivo {file_path} no existe")
    print("🔍 Buscando archivo en ubicaciones alternativas...")
    
    # Intentar rutas alternativas
    alternative_paths = [
        current_dir.parent.parent.parent / "data" / "scraper_enrichment" / "enriched_data.jsonl",
        current_dir.parent.parent.parent.parent / "scripts" / "data" / "scraper_enrichment" / "enriched_data.jsonl",
        Path("scripts/data/scraper_enrichment/enriched_data.jsonl")
    ]
    
    for alt_path in alternative_paths:
        if alt_path.exists():
            file_path = alt_path
            print(f"✅ Encontrado en: {file_path}")
            break
    else:
        print("❌ No se pudo encontrar el archivo enriched_data.jsonl")
        exit()

# Cargar datos desde el archivo JSONL
print(f"🔄 Cargando datos desde: {file_path}")
entidades_por_tipo = load_entities_from_jsonl(file_path)

# Mostrar estadísticas de entidades por tipo
print(f"\n�� ESTADÍSTICAS POR TIPO:")
for tipo, entidades in entidades_por_tipo.items():
    print(f"   • {tipo}: {len(entidades)} entidades")

# Ejecutar la función
find_similar_entities_with_urls(entidades_por_tipo, threshold=0.75)

�� Directorio actual: d:\TravelApp\Project\scripts\scraper_enrichment\manual_postprocessing
📄 Archivo objetivo: d:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data.jsonl
🔄 Cargando datos desde: d:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data.jsonl

�� ESTADÍSTICAS POR TIPO:
   • site: 805 entidades
   • transport: 41 entidades
   • itinerary: 69 entidades
   • desconocido: 387 entidades
   • tip: 119 entidades
   • event: 20 entidades
   • activity: 41 entidades
   • country: 18 entidades
   • province: 16 entidades
=== ENTIDADES SIMILARES (similitud >= 0.75) ===
Total de entidades: 1475
Grupos de entidades similares: 102

📍 7-Day Itinerary Option 1: Explore Bangkok and Northern Temples (principal) - Tipo: itinerary
   📍 URL: https://mundo-nomada.com/tailandia/que-ver-en-tailandia-en-7-dias/
   📰 Fuente: mundo-nomada.com
   └─ 7-Day Itinerary Option 3: From Bangkok to Northern Thailand (similitud: 0.78) - Tipo: itinerary
      �� URL: https://mundo-nom

## We check the normalized_entites integrity

In [60]:
import json
import ast
from pathlib import Path
from collections import defaultdict

def verificar_normalized_entities():
    """Verifica la estructura y contenido del archivo normalized_entities.py"""
    
    # Configurar rutas
    current_dir = Path.cwd()
    file_path = current_dir.parent.parent / "scraper_enrichment" / "auxs" / "normalized_entities.py"
    
    print("🔍 VERIFICACIÓN DE NORMALIZED_ENTITIES.PY")
    print("=" * 60)
    print(f"📄 Archivo: {file_path}")
    print(f" Directorio actual: {current_dir}")
    print()
    
    # Verificar si el archivo existe
    if not file_path.exists():
        print("❌ ERROR: El archivo normalized_entities.py no existe")
        return False
    
    print("✅ El archivo existe")
    print()
    
    # Leer el contenido del archivo
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        print("✅ Archivo leído correctamente")
    except Exception as e:
        print(f"❌ ERROR al leer el archivo: {e}")
        return False
    
    # Verificar si es un diccionario Python válido
    print("\n🔧 VERIFICACIÓN 1: ESTRUCTURA JSON/DICCIÓNARIO")
    print("-" * 50)
    
    try:
        # Buscar el diccionario en el archivo
        if 'NORMALIZED_ENTITIES = {' in content:
            # Extraer el diccionario
            start = content.find('NORMALIZED_ENTITIES = {')
            # Encontrar el final del diccionario (buscar el } que cierra el diccionario)
            brace_count = 0
            end = start
            
            for i, char in enumerate(content[start:], start):
                if char == '{':
                    brace_count += 1
                elif char == '}':
                    brace_count -= 1
                    if brace_count == 0:
                        end = i + 1
                        break
            
            dict_content = content[start:end]
            
            # Limpiar el contenido para evaluarlo
            dict_content = dict_content.replace('NORMALIZED_ENTITIES = ', '')
            
            # Evaluar el diccionario
            normalized_entities = ast.literal_eval(dict_content)
            print("✅ Estructura de diccionario válida")
            print(f" Total de entradas: {len(normalized_entities)}")
            
        else:
            print("❌ ERROR: No se encontró 'NORMALIZED_ENTITIES = {' en el archivo")
            return False
            
    except Exception as e:
        print(f"❌ ERROR: El archivo no tiene una estructura de diccionario válida: {e}")
        print(f"   Línea problemática aproximada: {dict_content[:200]}...")
        return False
    
    # Verificar que todas las claves están en los valores
    print("\n🔍 VERIFICACIÓN 2: CLAVES EN VALORES")
    print("-" * 50)
    
    claves_no_en_valores = []
    claves_en_valores = []
    
    for clave, valores in normalized_entities.items():
        if isinstance(valores, list):
            if clave in valores:
                claves_en_valores.append(clave)
            else:
                claves_no_en_valores.append(clave)
        else:
            print(f"⚠️  ADVERTENCIA: '{clave}' no tiene una lista de valores")
    
    if claves_en_valores:
        print(f"✅ {len(claves_en_valores)} claves SÍ están en sus valores:")
        for clave in claves_en_valores:
            print(f"   • {clave}")
    
    if claves_no_en_valores:
        print(f"❌ {len(claves_no_en_valores)} claves NO están en sus valores:")
        for clave in claves_no_en_valores:
            print(f"   • {clave}")
    
    # Verificar valores duplicados entre diferentes claves
    print("\n🔄 VERIFICACIÓN 3: VALORES DUPLICADOS ENTRE CLAVES")
    print("-" * 50)
    
    todos_valores = []
    valores_por_clave = {}
    
    for clave, valores in normalized_entities.items():
        if isinstance(valores, list):
            valores_por_clave[clave] = valores
            todos_valores.extend(valores)
    
    # Encontrar valores que aparecen en múltiples claves
    valores_duplicados = defaultdict(list)
    for valor in set(todos_valores):
        claves_que_contienen = [clave for clave, valores in valores_por_clave.items() if valor in valores]
        if len(claves_que_contienen) > 1:
            valores_duplicados[valor] = claves_que_contienen
    
    if valores_duplicados:
        print(f"⚠️  ADVERTENCIA: {len(valores_duplicadas)} valores aparecen en múltiples claves:")
        for valor, claves in valores_duplicados.items():
            print(f"   • '{valor}' aparece en: {', '.join(claves)}")
    else:
        print("✅ No hay valores duplicados entre diferentes claves")
    
    # Verificar claves duplicadas
    print("\n🔑 VERIFICACIÓN 4: CLAVES DUPLICADAS")
    print("-" * 50)
    
    claves_lista = list(normalized_entities.keys())
    claves_duplicadas = [clave for clave in set(claves_lista) if claves_lista.count(clave) > 1]
    
    if claves_duplicadas:
        print(f"❌ ERROR: {len(claves_duplicadas)} claves están duplicadas:")
        for clave in claves_duplicadas:
            print(f"   • {clave}")
    else:
        print("✅ No hay claves duplicadas")
    
    # Resumen final
    print("\n📋 RESUMEN FINAL")
    print("=" * 60)
    
    total_entradas = len(normalized_entities)
    claves_ok = len(claves_en_valores)
    claves_problema = len(claves_no_en_valores)
    valores_duplicados_count = len(valores_duplicados)
    claves_duplicadas_count = len(claves_duplicadas)
    
    print(f" Total de entradas: {total_entradas}")
    print(f"✅ Claves correctas: {claves_ok}")
    print(f"❌ Claves con problemas: {claves_problema}")
    print(f"⚠️  Valores duplicados: {valores_duplicados_count}")
    print(f"❌ Claves duplicadas: {claves_duplicadas_count}")
    
    # Estado general
    if claves_problema == 0 and claves_duplicadas_count == 0:
        print("\n🎉 ¡ESTADO PERFECTO! El archivo está bien estructurado")
        return True
    elif claves_problema == 0:
        print("\n⚠️  ADVERTENCIA: Hay claves duplicadas pero la estructura es correcta")
        return False
    else:
        print("\n❌ PROBLEMAS DETECTADOS: Revisar el archivo")
        return False

# Ejecutar la verificación
resultado = verificar_normalized_entities()

🔍 VERIFICACIÓN DE NORMALIZED_ENTITIES.PY
📄 Archivo: d:\TravelApp\Project\scripts\scraper_enrichment\auxs\normalized_entities.py
 Directorio actual: d:\TravelApp\Project\scripts\scraper_enrichment\manual_postprocessing

✅ El archivo existe

✅ Archivo leído correctamente

🔧 VERIFICACIÓN 1: ESTRUCTURA JSON/DICCIÓNARIO
--------------------------------------------------
✅ Estructura de diccionario válida
 Total de entradas: 34

🔍 VERIFICACIÓN 2: CLAVES EN VALORES
--------------------------------------------------
✅ 34 claves SÍ están en sus valores:
   • Bangkok
   • Chiang Mai
   • Phuket
   • Koh Samui
   • Koh Phi Phi
   • Similan
   • Mae Hong Son
   • Songthaew
   • Malasia
   • Ayutthaya
   • Damnoen Saduak Floating Market
   • minivan
   • Night Bazaar
   • tuk-tuk
   • Wat Rong Khun
   • Chatuchak Market
   • Triángulo de Oro
   • Grand Palace of Bangkok
   • Chiang Rai
   • Krabi
   • Pattaya
   • Hua Hin
   • Koh Phangan
   • Koh Tao
   • Phra Nang
   • Kanchanaburi
   • Koh Chang

## We group the entities with same name

In [61]:
import json
import ast
from pathlib import Path
from collections import defaultdict, Counter

# === 1. Cargar el diccionario de normalización ===
def load_normalized_entities(py_path):
    with open(py_path, 'r', encoding='utf-8') as f:
        content = f.read()
    start = content.find('NORMALIZED_ENTITIES = {')
    if start == -1:
        raise ValueError("No se encontró NORMALIZED_ENTITIES en el archivo.")
    brace_count = 0
    end = start
    for i, char in enumerate(content[start:], start):
        if char == '{':
            brace_count += 1
        elif char == '}':
            brace_count -= 1
            if brace_count == 0:
                end = i + 1
                break
    dict_content = content[start:end].replace('NORMALIZED_ENTITIES = ', '')
    return ast.literal_eval(dict_content)

# === 2. Cargar las entidades enriquecidas ===
def load_enriched_entities(jsonl_path):
    entities = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data = json.loads(line)
                url = data.get('url')
                for ent in data.get('entities', []):
                    ent['source_url'] = url
                    entities.append(ent)
    return entities

# === 3. Normalizar el nombre de la entidad ===
def normalize_entity_name(name, normalized_dict):
    if not name:
        return ""
    for norm, variations in normalized_dict.items():
        if name.lower() in [v.lower() for v in variations]:
            return norm
    return name

# === 4. Agrupar entidades (CORREGIDO) ===
def get_key(ent):
    # Agrupar por: tipo, subtipo, ciudad, nombre normalizado
    # PERO NO por descripción
    tipo = (ent.get('entity_type') or '').lower()
    subtipo = (ent.get('subtype') or '').lower()
    
    # Buscar ciudad en la jerarquía
    ciudad = None
    for h in ent.get('hierarchy', []):
        if (h.get('type') or '').lower() == 'city':
            ciudad = (h.get('name') or '').lower()
            break
    
    # Si no hay jerarquía, buscar en otros campos
    if not ciudad:
        ciudad = (ent.get('city') or '').lower()
    
    nombre = (ent.get('normalized_name') or '').lower()
    
    return (tipo, subtipo, ciudad, nombre)

def agrupar_entidades(entities, normalized_dict):
    agrupadas = defaultdict(list)
    for ent in entities:
        # Normalizar nombre
        original_name = ent.get('name') or ent.get('title') or ent.get('advice_text')
        ent['normalized_name'] = normalize_entity_name(original_name, normalized_dict)
        key = get_key(ent)
        agrupadas[key].append(ent)
    return agrupadas

# === 5. Generar la salida (MEJORADO) ===
def generar_salida(agrupadas, output_path):
    output = []
    for key, ents in agrupadas.items():
        if not key[3]:  # Saltar entidades sin nombre
            continue
            
        # Tomar la primera entidad como base
        base = dict(ents[0])
        base['appearances'] = len(ents)
        base['all_source_urls'] = list({e.get('source_url') for e in ents if e.get('source_url')})
        
        # COMBINAR DESCRIPCIONES (esto es lo que se combina, no se usa para distinguir)
        descripciones = []
        for ent in ents:
            desc = ent.get('description', '').strip()
            if desc and desc not in descripciones:
                descripciones.append(desc)
        
        if descripciones:
            if len(descripciones) == 1:
                base['description'] = descripciones[0]
            else:
                base['description'] = " | ".join(descripciones)
        
        # COMBINAR IMÁGENES
        todas_imagenes = []
        for ent in ents:
            imagenes = ent.get('images', [])
            if isinstance(imagenes, list):
                todas_imagenes.extend(imagenes)
        
        if todas_imagenes:
            base['images'] = list(set(todas_imagenes))  # Eliminar duplicados
        
        # COMBINAR WEBSITES OFICIALES
        websites = []
        for ent in ents:
            website = ent.get('official_website', '').strip()
            if website and website not in websites:
                websites.append(website)
        
        if websites:
            base['official_websites'] = websites
        
        output.append(base)
    
    # Guardar en JSONL
    with open(output_path, 'w', encoding='utf-8') as f:
        for ent in output:
            f.write(json.dumps(ent, ensure_ascii=False) + '\n')
    return output

# === 6. Estadísticas y prints bonitos ===
def print_stats(entities, agrupadas, output):
    print("🔎 Estadísticas del proceso de agrupación")
    print("="*60)
    print(f"Total de entidades originales: {len(entities)}")
    print(f"Total de grupos únicos tras agrupar: {len(agrupadas)}")
    print(f"Total de entidades en el archivo final: {len(output)}")
    print("\nEjemplo de grupo agrupado:")
    for k, v in agrupadas.items():
        if k[3]:  # Solo mostrar grupos con nombre
            print(f"  - Clave: {k}")
            print(f"    Nombres originales: {[e.get('name') for e in v]}")
            print(f"    Aparece en URLs: {[e.get('source_url') for e in v]}")
            print(f"    Número de apariciones: {len(v)}")
            print()
            break
    apariciones = [ent['appearances'] for ent in output]
    print(f"Media de apariciones por grupo: {sum(apariciones)/len(apariciones):.2f}")
    print(f"Máximo de apariciones en un grupo: {max(apariciones)}")
    print(f"Mínimo de apariciones en un grupo: {min(apariciones)}")
    print("="*60)

# === 7. Rutas ===
input_jsonl = Path(r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data.jsonl")
normalized_py = Path(r"D:\TravelApp\Project\scripts\scraper_enrichment\auxs\normalized_entities.py")
output_jsonl = Path(r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_2.jsonl")

# === 8. Proceso completo ===
normalized_dict = load_normalized_entities(normalized_py)
entities = load_enriched_entities(input_jsonl)
agrupadas = agrupar_entidades(entities, normalized_dict)
output = generar_salida(agrupadas, output_jsonl)
print_stats(entities, agrupadas, output)
print(f"\n✅ Archivo guardado en: {output_jsonl}")

🔎 Estadísticas del proceso de agrupación
Total de entidades originales: 1516
Total de grupos únicos tras agrupar: 1070
Total de entidades en el archivo final: 1056

Ejemplo de grupo agrupado:
  - Clave: ('site', 'country', '', 'thailand')
    Nombres originales: ['Tailandia', 'Thailand', 'Thailand', 'Tailandia', 'Tailandia', 'Thailand', 'Thailand', 'Tailandia', 'Tailandia', 'Tailandia', 'Tailandia', 'Tailandia', 'Tailandia', 'Tailandia', 'Tailandia', 'Tailandia', 'Thailand', 'Thailand', 'Tailandia', 'Tailandia', 'Thailand', 'Thailand', 'Thailand', 'Thailand', 'Thailand', 'Thailand']
    Aparece en URLs: ['https://www.viajeroscallejeros.com/lugares-que-visitar-en-tailandia/', 'https://www.viajeroscallejeros.com/lugares-que-visitar-en-tailandia/', 'https://heymondo.es/blog/que-ver-tailandia/', 'https://www.mundoasiatours.com/que-ver-en-tailandia/', 'https://www.expansion.com/fueradeserie/viajes/2023/09/05/64f5b529e5fdeaa9778b45f4.html', 'https://lacosmopolilla.com/que-ver-en-tailandia/',

## We show the data

In [62]:
import json
from pathlib import Path
from collections import Counter

def imprimir_entidades_agrupadas(output_path, max_mostrar=10):
    """
    Imprime información detallada de las entidades agrupadas
    """
    print("🔍 IMPRESOR DE ENTIDADES AGRUPADAS")
    print("=" * 80)
    
    # Cargar el archivo de salida
    with open(output_path, 'r', encoding='utf-8') as f:
        entidades = [json.loads(line) for line in f if line.strip()]
    
    print(f"�� Total de entidades agrupadas: {len(entidades)}")
    print()
    
    # Estadísticas generales
    tipos_entidad = Counter([e.get('entity_type', 'sin_tipo') for e in entidades])
    print("�� DISTRIBUCIÓN POR TIPO DE ENTIDAD:")
    for tipo, count in tipos_entidad.most_common():
        print(f"   • {tipo}: {count}")
    print()
    
    # Entidades con más apariciones
    entidades_por_apariciones = sorted(entidades, key=lambda x: x.get('appearances', 0), reverse=True)
    print(f"🏆 TOP {max_mostrar} ENTIDADES CON MÁS APARICIONES:")
    print("-" * 80)
    
    for i, ent in enumerate(entidades_por_apariciones[:max_mostrar], 1):
        print(f"{i:2d}. {ent.get('normalized_name', ent.get('name', 'Sin nombre'))}")
        print(f"     Tipo: {ent.get('entity_type', 'N/A')}")
        print(f"     Subtipo: {ent.get('subtype', 'N/A')}")
        print(f"     Apariciones: {ent.get('appearances', 0)}")
        print(f"     URLs: {len(ent.get('all_source_urls', []))}")
        if ent.get('description'):
            desc = ent['description'][:100] + "..." if len(ent['description']) > 100 else ent['description']
            print(f"     Descripción: {desc}")
        print()
    
    # Entidades únicas (solo aparecen una vez)
    entidades_unicas = [e for e in entidades if e.get('appearances', 0) == 1]
    print(f"⭐ ENTIDADES ÚNICAS (solo 1 aparición): {len(entidades_unicas)}")
    print("-" * 80)
    
    for i, ent in enumerate(entidades_unicas[:5], 1):  # Mostrar solo las primeras 5
        print(f"{i}. {ent.get('normalized_name', ent.get('name', 'Sin nombre'))} - {ent.get('entity_type', 'N/A')}")
    
    if len(entidades_unicas) > 5:
        print(f"   ... y {len(entidades_unicas) - 5} más")
    print()
    
    # Ejemplos de grupos con múltiples apariciones
    grupos_multiples = [e for e in entidades if e.get('appearances', 0) > 1]
    print(f"🔄 GRUPOS CON MÚLTIPLES APARICIONES: {len(grupos_multiples)}")
    print("-" * 80)
    
    for i, ent in enumerate(grupos_multiples[:3], 1):  # Mostrar solo los primeros 3
        print(f"{i}. {ent.get('normalized_name', ent.get('name', 'Sin nombre'))}")
        print(f"   Apariciones: {ent.get('appearances', 0)}")
        print(f"   URLs de origen:")
        for url in ent.get('all_source_urls', [])[:3]:  # Mostrar solo las primeras 3 URLs
            print(f"     • {url}")
        if len(ent.get('all_source_urls', [])) > 3:
            print(f"     • ... y {len(ent.get('all_source_urls', [])) - 3} más")
        print()
    
    # Verificar normalización
    print("🔧 VERIFICACIÓN DE NORMALIZACIÓN:")
    print("-" * 80)
    
    entidades_sin_normalizar = [e for e in entidades if e.get('normalized_name') == e.get('name')]
    entidades_normalizadas = [e for e in entidades if e.get('normalized_name') != e.get('name')]
    
    print(f"   • Entidades normalizadas: {len(entidades_normalizadas)}")
    print(f"   • Entidades sin cambios: {len(entidades_sin_normalizar)}")
    
    if entidades_normalizadas:
        print("   Ejemplos de normalización:")
        for ent in entidades_normalizadas[:3]:
            print(f"     • '{ent.get('name')}' → '{ent.get('normalized_name')}'")
    print()
    
    # Resumen final
    print("📋 RESUMEN FINAL")
    print("=" * 80)
    print(f"✅ Total entidades procesadas: {len(entidades)}")
    print(f"✅ Entidades únicas: {len(entidades_unicas)}")
    print(f"✅ Grupos múltiples: {len(grupos_multiples)}")
    print(f"✅ Tipos diferentes: {len(tipos_entidad)}")
    print(f"✅ Entidades normalizadas: {len(entidades_normalizadas)}")
    
    # Calcular estadísticas de apariciones
    apariciones = [e.get('appearances', 0) for e in entidades]
    if apariciones:
        print(f"✅ Media apariciones: {sum(apariciones)/len(apariciones):.2f}")
        print(f"✅ Máximo apariciones: {max(apariciones)}")
        print(f"✅ Mínimo apariciones: {min(apariciones)}")

# Ejecutar el impresor
output_path = Path(r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_2.jsonl")
imprimir_entidades_agrupadas(output_path, max_mostrar=15)

🔍 IMPRESOR DE ENTIDADES AGRUPADAS
�� Total de entidades agrupadas: 1056

�� DISTRIBUCIÓN POR TIPO DE ENTIDAD:
   • site: 522
   • sin_tipo: 236
   • tip: 119
   • itinerary: 58
   • activity: 41
   • transport: 27
   • event: 19
   • country: 18
   • province: 16

🏆 TOP 15 ENTIDADES CON MÁS APARICIONES:
--------------------------------------------------------------------------------
 1. Thailand
     Tipo: site
     Subtipo: country
     Apariciones: 26
     URLs: 19
     Descripción: Diverse country with attractions for all tastes, known for its hospitality, delicious cuisine, and b...

 2. Chiang Mai
     Tipo: site
     Subtipo: city
     Apariciones: 17
     URLs: 12
     Descripción: Known as the 'Northern Capital' of Thailand, offering a magical environment with nature, Buddhist te...

 3. Phuket
     Tipo: site
     Subtipo: island
     Apariciones: 16
     URLs: 14
     Descripción: Popular island destination in Thailand. | Phuket is the largest island in Thailand, known for it

## We do manual modification of the entities

In [67]:
# === BLOQUE PARA JUPYTER NOTEBOOK ===
import sys
from pathlib import Path

# Añadir el directorio del script al path
current_dir = Path.cwd()
script_dir = current_dir / "entidades_editor.py"
sys.path.append(str(current_dir))

# Importar la clase
from entidades_editor import iniciar_editor

# Definir rutas
input_path = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl"
output_path = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl"

print("🚀 Iniciando Editor de Entidades...")
print(f"📥 Entrada: {input_path}")
print(f"📤 Salida: {output_path}")
print()

# Iniciar el editor
iniciar_editor(input_path, output_path)

🚀 Iniciando Editor de Entidades...
📥 Entrada: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
📤 Salida: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl

✅ Usando archivo existente: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
✅ Cargadas 1051 entidades desde D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
✅ Entidades guardadas en: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
✅ OSM añadido: OSM:N2082718893
✅ Wikidata añadido: Q669271
✅ Entidades guardadas en: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
✅ Entidades guardadas en: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
✅ Entidades guardadas en: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
✅ Entidades guardadas en: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
✅ Entidades guardad

## Llamas a API de NOMINATIN

In [4]:
# === BLOQUE PARA JUPYTER NOTEBOOK ===
import sys
from pathlib import Path

# Añadir el directorio del script al path
current_dir = Path.cwd()
script_dir = current_dir / "nominatim_enricher.py"
sys.path.append(str(current_dir))

# Importar las funciones
from nominatim_enricher import process_sites_with_nominatim, show_sample_results

# Definir rutas
input_path = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl"
output_path = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_4.jsonl"

print("�� INICIANDO ENRIQUECIMIENTO CON NOMINATIM")
print("=" * 60)
print(f"📥 Entrada: {input_path}")
print(f"📤 Salida: {output_path}")
print()

# Procesar todos los sitios
stats = process_sites_with_nominatim(input_path, output_path)

# Mostrar ejemplos de resultados
show_sample_results(output_path, num_samples=10)

print("\n🎯 RESUMEN FINAL:")
print(f"   • Total de sitios procesados: {stats['total_sites']}")
print(f"   • Sitios encontrados: {stats['found_sites']}")
print(f"   • Tasa de éxito: {stats['success_rate']*100:.1f}%")
print(f"   • Distribución por calidad: {stats['quality_stats']}")

�� INICIANDO ENRIQUECIMIENTO CON NOMINATIM
📥 Entrada: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
📤 Salida: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_4.jsonl

🚀 INICIANDO ENRIQUECIMIENTO CON NOMINATIM
📥 Entrada: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
📤 Salida: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_4.jsonl

📊 Estadísticas:
   • Total de entidades: 1049
   • Sitios a procesar: 520

🔄 Procesando sitio 1/520: Tailandia
🔍 Buscando: 'Tailandia' (name_only)
✅ Encontrado: Tailândia, Região Geográfica Imediata de Abaetetuba, Região Geográfica Intermediária de Belém, Pará, Região Norte, 68695-000, Brasil (score: 0.44, nivel: poor)

🔄 Procesando sitio 2/520: Bangkok
🔍 Buscando: 'Bangkok' (name_only)
✅ Encontrado: กรุงเทพมหานคร, ประเทศไทย (score: 0.72, nivel: good)

🔄 Procesando sitio 3/520: Chiang Mai
🔍 Buscando: 'Chiang Mai' (name_only)
✅ Encontrado: เทศบาลนครเชียงใหม่, ฟ้าฮ่า

## We short the elements by the number of appearences

In [20]:
import json
from pathlib import Path
from collections import defaultdict

def ordenar_por_appearances(input_path, output_path):
    """Ordena las entidades por appearances de forma descendente"""
    
    print("🔄 ORDENANDO ENTIDADES POR APARICIONES")
    print("=" * 50)
    print(f"📥 Entrada: {input_path}")
    print(f"📤 Salida: {output_path}")
    print()
    
    # Cargar todas las entidades
    entidades = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                entidades.append(json.loads(line))
    
    print(f"�� Total de entidades cargadas: {len(entidades)}")
    
    # Ordenar por appearances (descendente)
    entidades_ordenadas = sorted(
        entidades, 
        key=lambda x: x.get('appearances', 0), 
        reverse=True
    )
    
    # Guardar ordenadas
    with open(output_path, 'w', encoding='utf-8') as f:
        for entidad in entidades_ordenadas:
            f.write(json.dumps(entidad, ensure_ascii=False) + '\n')
    
    print(f"✅ Entidades ordenadas y guardadas en: {output_path}")
    
    # Mostrar top 20
    print(f"\n🏆 TOP 20 POR APARICIONES:")
    print("-" * 50)
    
    for i, entidad in enumerate(entidades_ordenadas[:20], 1):
        nombre = entidad.get('name', 'Sin nombre')
        appearances = entidad.get('appearances', 0)
        tipo = entidad.get('entity_type', 'sin_tipo')
        subtipo = entidad.get('subtype', 'N/A')
        
        print(f"{i:2d}. {nombre}")
        print(f"     • Apariciones: {appearances}")
        print(f"     • Tipo: {tipo}")
        print(f"     • Subtipo: {subtipo}")
        
        # Mostrar datos de Nominatim si existen
        nominatim_data = entidad.get('nominatim_match', {})
        if nominatim_data.get('found'):
            quality = nominatim_data.get('quality_level', 'N/A')
            score = nominatim_data.get('quality_score', 0)
            print(f"     • Nominatim: {quality} ({score:.2f})")
        
        print()
    
    # Estadísticas generales
    total_appearances = sum(e.get('appearances', 0) for e in entidades_ordenadas)
    avg_appearances = total_appearances / len(entidades_ordenadas) if entidades_ordenadas else 0
    
    print(f"📈 ESTADÍSTICAS:")
    print(f"   • Total de apariciones: {total_appearances}")
    print(f"   • Promedio de apariciones: {avg_appearances:.1f}")
    print(f"   • Máximo de apariciones: {entidades_ordenadas[0].get('appearances', 0) if entidades_ordenadas else 0}")
    print(f"   • Mínimo de apariciones: {entidades_ordenadas[-1].get('appearances', 0) if entidades_ordenadas else 0}")
    
    return {
        'total_entidades': len(entidades_ordenadas),
        'total_appearances': total_appearances,
        'max_appearances': entidades_ordenadas[0].get('appearances', 0) if entidades_ordenadas else 0,
        'min_appearances': entidades_ordenadas[-1].get('appearances', 0) if entidades_ordenadas else 0
    }

# Ejecutar ordenamiento
input_file = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl"
output_file = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl"

# Verificar que existe el archivo de entrada
input_path = Path(input_file)
if not input_path.exists():
    print(f"❌ ERROR: No existe el archivo: {input_file}")
    print("�� Asegúrate de haber ejecutado el editor primero")
else:
    print(f"✅ Archivo encontrado: {input_path.name}")
    stats = ordenar_por_appearances(input_file, output_file)
    print(f"\n🎉 ¡Ordenamiento completado!")

✅ Archivo encontrado: enriched_data_5.jsonl
🔄 ORDENANDO ENTIDADES POR APARICIONES
📥 Entrada: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl
📤 Salida: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl

�� Total de entidades cargadas: 481
✅ Entidades ordenadas y guardadas en: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl

🏆 TOP 20 POR APARICIONES:
--------------------------------------------------
 1. Chiang Rai
     • Apariciones: 5
     • Tipo: site
     • Subtipo: city
     • Nominatim: poor (0.47)

 2. Sukhothai
     • Apariciones: 4
     • Tipo: site
     • Subtipo: city
     • Nominatim: poor (0.30)

 3. Krabi
     • Apariciones: 4
     • Tipo: site
     • Subtipo: city
     • Nominatim: poor (0.44)

 4. Pai
     • Apariciones: 4
     • Tipo: sin_tipo
     • Subtipo: city

 5. Doi Inthanon
     • Apariciones: 4
     • Tipo: site
     • Subtipo: mountain
     • Nominatim: poor (0.42)

 6. Koh Lipe
    

## We match the entities by name and subtype

In [12]:
import json
from pathlib import Path
from collections import defaultdict

def agrupar_por_nombre_subtipo(input_path, output_path):
    """Agrupa entidades que tengan el mismo nombre y subtipo"""
    
    print("🔄 AGRUPANDO ENTIDADES POR NOMBRE Y SUBTIPO")
    print("=" * 50)
    print(f"📥 Entrada: {input_path}")
    print(f"📤 Salida: {output_path}")
    print()
    
    # Cargar todas las entidades
    entidades = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                entidades.append(json.loads(line))
    
    print(f"�� Total de entidades cargadas: {len(entidades)}")
    
    # Agrupar por nombre y subtipo
    grupos = defaultdict(list)
    
    for entidad in entidades:
        nombre = entidad.get('name', '').strip().lower()
        subtipo = entidad.get('subtype', '').strip().lower()
        
        if nombre and subtipo:
            clave = (nombre, subtipo)
            grupos[clave].append(entidad)
    
    # Crear entidades agrupadas
    entidades_agrupadas = []
    grupos_procesados = 0
    
    for (nombre, subtipo), entidades_grupo in grupos.items():
        if len(entidades_grupo) > 1:
            # Hay múltiples entidades para agrupar
            print(f"🔄 Agrupando: '{nombre}' (subtipo: {subtipo}) - {len(entidades_grupo)} entidades")
            
            # Tomar la primera entidad como base
            base = dict(entidades_grupo[0])
            
            # Combinar datos
            descripciones = []
            todas_imagenes = []
            websites = []
            todas_urls = []
            source_urls = []
            
            # Procesar todas las entidades del grupo
            for entidad in entidades_grupo:
                # Descripciones
                desc = entidad.get('description', '').strip()
                if desc and desc not in descripciones:
                    descripciones.append(desc)
                
                # Imágenes
                imagenes = entidad.get('images', [])
                if isinstance(imagenes, list):
                    todas_imagenes.extend(imagenes)
                
                # Websites
                website = entidad.get('official_website', '').strip()
                if website and website not in websites:
                    websites.append(website)
                
                # URLs de origen
                url = entidad.get('source_url', '').strip()
                if url and url not in todas_urls:
                    todas_urls.append(url)
                
                # URLs de origen (campo all_source_urls)
                all_urls = entidad.get('all_source_urls', [])
                if isinstance(all_urls, list):
                    source_urls.extend(all_urls)
            
            # Actualizar la entidad base
            if descripciones:
                base['description'] = " | ".join(descripciones)
            
            if todas_imagenes:
                base['images'] = list(set(todas_imagenes))  # Eliminar duplicados
            
            if websites:
                base['official_website'] = " | ".join(websites)
            
            if todas_urls:
                base['source_url'] = todas_urls[0]  # Mantener la primera como principal
                base['all_source_urls'] = list(set(source_urls + todas_urls))  # Combinar todas
            
            # Añadir contador de apariciones
            base['appearances'] = len(entidades_grupo)
            
            # Añadir información de agrupación
            base['agrupado_por'] = f"nombre_subtipo: {nombre}_{subtipo}"
            base['entidades_originales'] = len(entidades_grupo)
            
            entidades_agrupadas.append(base)
            grupos_procesados += 1
            
        else:
            # Solo una entidad, añadir sin cambios
            entidades_agrupadas.append(entidades_grupo[0])
    
    # Guardar resultados
    with open(output_path, 'w', encoding='utf-8') as f:
        for entidad in entidades_agrupadas:
            f.write(json.dumps(entidad, ensure_ascii=False) + '\n')
    
    # Estadísticas
    print(f"\n�� AGRUPACIÓN COMPLETADA")
    print("=" * 50)
    print(f"📊 Estadísticas:")
    print(f"   • Entidades originales: {len(entidades)}")
    print(f"   • Entidades finales: {len(entidades_agrupadas)}")
    print(f"   • Grupos procesados: {grupos_procesados}")
    print(f"   • Entidades eliminadas: {len(entidades) - len(entidades_agrupadas)}")
    print(f"   • Reducción: {((len(entidades) - len(entidades_agrupadas)) / len(entidades) * 100):.1f}%")
    
    # Mostrar ejemplos de agrupaciones
    print(f"\n📋 EJEMPLOS DE AGRUPACIONES:")
    print("-" * 50)
    
    for i, entidad in enumerate(entidades_agrupadas[:10], 1):
        nombre = entidad.get('name', 'Sin nombre')
        subtipo = entidad.get('subtype', 'N/A')
        appearances = entidad.get('appearances', 1)
        
        if appearances > 1:
            print(f"{i}. {nombre} ({subtipo})")
            print(f"   • Apariciones: {appearances}")
            print(f"   • Descripción: {entidad.get('description', 'N/A')[:100]}...")
            print()
    
    return {
        'entidades_originales': len(entidades),
        'entidades_finales': len(entidades_agrupadas),
        'grupos_procesados': grupos_procesados,
        'reduccion_porcentual': ((len(entidades) - len(entidades_agrupadas)) / len(entidades) * 100)
    }

# Ejecutar agrupación
input_file = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl"
output_file = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl"

# Verificar que existe el archivo de entrada
input_path = Path(input_file)
if not input_path.exists():
    print(f"❌ ERROR: No existe el archivo: {input_file}")
    print(" Asegúrate de haber ejecutado el ordenamiento primero")
else:
    print(f"✅ Archivo encontrado: {input_path.name}")
    stats = agrupar_por_nombre_subtipo(input_file, output_file)
    print(f"\n🎉 ¡Agrupación completada!")

✅ Archivo encontrado: enriched_data_5.jsonl
🔄 AGRUPANDO ENTIDADES POR NOMBRE Y SUBTIPO
📥 Entrada: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl
📤 Salida: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl

�� Total de entidades cargadas: 1015
🔄 Agrupando: 'thailand' (subtipo: country) - 2 entidades
🔄 Agrupando: 'phuket' (subtipo: island) - 2 entidades
🔄 Agrupando: 'bangkok' (subtipo: city) - 2 entidades
🔄 Agrupando: 'koh samui' (subtipo: island) - 2 entidades
🔄 Agrupando: 'koh tao' (subtipo: island) - 2 entidades
🔄 Agrupando: 'krabi' (subtipo: province) - 2 entidades
🔄 Agrupando: 'ayutthaya' (subtipo: city) - 2 entidades
🔄 Agrupando: 'sukhothai' (subtipo: city) - 4 entidades
🔄 Agrupando: 'chiang rai' (subtipo: city) - 5 entidades
🔄 Agrupando: 'chiang mai' (subtipo: city) - 3 entidades
🔄 Agrupando: 'krabi' (subtipo: city) - 4 entidades
🔄 Agrupando: 'islas phi phi' (subtipo: island) - 2 entidades
🔄 Agrupando: 'koh chang' (subtipo: isla

In [25]:
import json
from pathlib import Path
from collections import defaultdict

def calcular_apariciones_desde_4(input_path_4):
    """Calcula apariciones usando el campo appearances existente"""
    
    print("🔄 CALCULANDO APARICIONES DESDE ENRICHED_DATA_4.JSONL")
    print("=" * 60)
    print(f"📥 Entrada: {input_path_4}")
    print()
    
    # Cargar entidades del archivo 4
    entidades = []
    with open(input_path_4, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                entidades.append(json.loads(line))
    
    print(f" Entidades cargadas: {len(entidades)}")
    
    # Agrupar por nombre y subtipo, SUMANDO appearances existentes
    apariciones_por_entidad = defaultdict(int)
    
    for entidad in entidades:
        nombre = entidad.get('name', '').strip().lower()
        subtipo = entidad.get('subtype', '').strip().lower()
        appearances = entidad.get('appearances', 1)  # ← USAR EL CAMPO EXISTENTE
        
        if nombre and subtipo:
            clave = (nombre, subtipo)
            apariciones_por_entidad[clave] += appearances  # ← SUMAR appearances
    
    print(f"📈 Entidades únicas encontradas: {len(apariciones_por_entidad)}")
    
    # Mostrar top 10 por apariciones
    print(f"\n🏆 TOP 10 POR APARICIONES:")
    print("-" * 50)
    
    sorted_apariciones = sorted(apariciones_por_entidad.items(), key=lambda x: x[1], reverse=True)
    
    for i, ((nombre, subtipo), count) in enumerate(sorted_apariciones[:10], 1):
        print(f"{i:2d}. {nombre} ({subtipo}): {count} apariciones")
    
    return apariciones_por_entidad
def actualizar_y_ordenar_5(input_path_5, output_path_5, estructura_apariciones):
    """Actualiza apariciones en 5 y ordena por apariciones descendentes"""
    
    print(f"\n🔄 ACTUALIZANDO Y ORDENANDO ENRICHED_DATA_5.JSONL")
    print("=" * 60)
    print(f"📥 Entrada: {input_path_5}")
    print(f"📤 Salida: {output_path_5}")
    print()
    
    # Cargar entidades del archivo 5
    entidades_actualizadas = []
    
    with open(input_path_5, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                entidad = json.loads(line)
                
                # Buscar apariciones para esta entidad
                nombre = entidad.get('name', '').strip().lower()
                subtipo = entidad.get('subtype', '').strip().lower()
                
                if nombre and subtipo:
                    clave = (nombre, subtipo)
                    apariciones_calculadas = estructura_apariciones.get(clave, 1)
                    
                    # Actualizar appearances
                    entidad['appearances'] = apariciones_calculadas
                    print(f"✅ {nombre} ({subtipo}): {apariciones_calculadas} apariciones")
                
                entidades_actualizadas.append(entidad)
    
    # ORDENAR por apariciones descendentes
    entidades_ordenadas = sorted(
        entidades_actualizadas, 
        key=lambda x: x.get('appearances', 0), 
        reverse=True
    )
    
    # Guardar archivo ordenado
    with open(output_path_5, 'w', encoding='utf-8') as f:
        for entidad in entidades_ordenadas:
            f.write(json.dumps(entidad, ensure_ascii=False) + '\n')
    
    print(f"\n📊 RESUMEN:")
    print(f"   • Entidades procesadas: {len(entidades_ordenadas)}")
    print(f"   • Archivo guardado: {output_path_5}")
    
    # Mostrar top 10 del archivo final
    print(f"\n🏆 TOP 10 FINAL:")
    print("-" * 50)
    
    for i, entidad in enumerate(entidades_ordenadas[:10], 1):
        nombre = entidad.get('name', 'Sin nombre')
        subtipo = entidad.get('subtype', 'N/A')
        appearances = entidad.get('appearances', 0)
        print(f"{i:2d}. {nombre} ({subtipo}): {appearances} apariciones")

# Ejecutar el proceso completo
input_file_4 = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_4.jsonl"
input_file_5 = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl"
output_file_5 = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl"

# Verificar archivos
path_4 = Path(input_file_4)
path_5 = Path(input_file_5)

if not path_4.exists():
    print(f"❌ ERROR: No existe enriched_data_4.jsonl")
elif not path_5.exists():
    print(f"❌ ERROR: No existe enriched_data_5.jsonl")
else:
    print("✅ Archivos encontrados, iniciando proceso...")
    
    # Paso 1: Calcular apariciones desde archivo 4
    estructura_apariciones = calcular_apariciones_desde_4(input_file_4)
    
    # Paso 2: Actualizar apariciones en archivo 5 y ordenar
    actualizar_y_ordenar_5(input_file_5, output_file_5, estructura_apariciones)
    
    print(f"\n🎉 ¡PROCESO COMPLETADO!")
    print(f"📁 Archivo actualizado y ordenado: {output_file_5}")

✅ Archivos encontrados, iniciando proceso...
🔄 CALCULANDO APARICIONES DESDE ENRICHED_DATA_4.JSONL
📥 Entrada: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_4.jsonl

 Entidades cargadas: 1049
📈 Entidades únicas encontradas: 488

🏆 TOP 10 POR APARICIONES:
--------------------------------------------------
 1. tailandia (country): 36 apariciones
 2. bangkok (city): 32 apariciones
 3. ayutthaya (city): 26 apariciones
 4. chiang rai (city): 25 apariciones
 5. koh samui (island): 23 apariciones
 6. phuket (island): 21 apariciones
 7. sukhothai (city): 19 apariciones
 8. koh lipe (island): 19 apariciones
 9. koh tao (island): 19 apariciones
10. krabi (city): 15 apariciones

🔄 ACTUALIZANDO Y ORDENANDO ENRICHED_DATA_5.JSONL
📥 Entrada: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl
📤 Salida: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl

✅ chiang rai (city): 25 apariciones
✅ sukhothai (city): 19 apariciones
✅ krabi (cit

## We manually modify the Nominatim API and the old data for the enriched database

In [30]:
# === BLOQUE PARA JUPYTER NOTEBOOK - EDITOR CON NOMINATIM ===
import sys
from pathlib import Path

# Añadir el directorio del script al path
current_dir = Path.cwd()
script_dir = current_dir / "entidades_editor.py"
sys.path.append(str(current_dir))

# Importar la clase
from entidades_editor import iniciar_editor

# Definir rutas para enriched_data_4.jsonl
input_path = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl"
output_path = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl"

print("🚀 Iniciando Editor de Entidades con Nominatim...")
print("=" * 60)
print(f"📥 Entrada: {input_path}")
print(f"📤 Salida: {output_path}")
print()

# Verificar que el archivo de entrada existe
input_file = Path(input_path)
if not input_file.exists():
    print(f"❌ ERROR: No existe el archivo de entrada: {input_path}")
    print("�� Asegúrate de haber ejecutado primero el enriquecimiento con Nominatim")
else:
    print(f"✅ Archivo de entrada encontrado: {input_file.name}")
    print(f"📊 Tamaño: {input_file.stat().st_size / 1024:.1f} KB")
    print()

# Iniciar el editor
print("��️ Abriendo interfaz gráfica...")
iniciar_editor(input_path, output_path)

print("\n🎯 FUNCIONALIDADES DISPONIBLES:")
print("   • Navegación: Primera, anterior, siguiente, última")
print("   • Edición: Modificar JSON con agrupamiento automático")
print("   • Eliminación: Eliminar entidades")
print("   • OSM/Wikidata: Añadir identificadores manualmente")
print("   • Nominatim: Ver datos de enriquecimiento")
print("   • Guardado automático: Se guarda tras cada cambio")
print("   • Copia automática: enriched_data_4.jsonl → enriched_data_5.jsonl")
print()
print("💡 CONSEJOS:")
print("   • Los sitios tienen campo 'nominatim_match' con datos de OSM")
print("   • Puedes verificar calidad de matches (excellent, good, fair, poor)")
print("   • Coordenadas y Wikipedia ya están incluidas si se encontraron")
print("   • Puedes añadir OSM/Wikidata manualmente si faltan")

🚀 Iniciando Editor de Entidades con Nominatim...
📥 Entrada: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl
📤 Salida: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl

✅ Archivo de entrada encontrado: enriched_data_5.jsonl
📊 Tamaño: 164.7 KB

��️ Abriendo interfaz gráfica...
✅ Usando archivo existente: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl
✅ Cargadas 124 entidades desde D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl
✅ Entidades guardadas en: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl
✅ Entidades guardadas en: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl
✅ Entidades guardadas en: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl
✅ Entidades guardadas en: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl
✅ Entidades guardadas en: D:\TravelApp\Project\scripts\dat

## We print all types and subtypes

In [34]:
import json
from pathlib import Path
from collections import defaultdict, Counter

def analizar_tipos_entidades(input_path):
    """Analiza y muestra todos los tipos y subtipos de entidades"""
    
    print("�� ANALIZANDO TIPOS Y SUBTIPOS DE ENTIDADES")
    print("=" * 60)
    print(f"📥 Archivo: {input_path}")
    print()
    
    # Cargar entidades
    entidades = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                entidades.append(json.loads(line))
    
    print(f" Entidades cargadas: {len(entidades)}")
    print()
    
    # Contar tipos y subtipos
    tipos_counter = Counter()
    subtipos_counter = Counter()
    combinaciones = defaultdict(int)
    
    for entidad in entidades:
        tipo = entidad.get('entity_type', 'sin_tipo')
        subtipo = entidad.get('subtype', 'sin_subtipo')
        
        tipos_counter[tipo] += 1
        subtipos_counter[subtipo] += 1
        combinaciones[f"{tipo}_{subtipo}"] += 1
    
    # Mostrar tipos principales
    print("��️  TIPOS DE ENTIDADES:")
    print("-" * 40)
    for tipo, count in tipos_counter.most_common():
        print(f"   • {tipo}: {count}")
    
    print()
    
    # Mostrar subtipos principales
    print("📋 SUBTIPOS DE ENTIDADES:")
    print("-" * 40)
    for subtipo, count in subtipos_counter.most_common():
        print(f"   • {subtipo}: {count}")
    
    print()
    
    # Mostrar combinaciones más frecuentes
    print("�� COMBINACIONES TIPO-SUBTIPO (TOP 20):")
    print("-" * 50)
    for combinacion, count in sorted(combinaciones.items(), key=lambda x: x[1], reverse=True)[:20]:
        tipo, subtipo = combinacion.split('_', 1)
        print(f"   • {tipo} + {subtipo}: {count}")
    
    print()
    
    # Estadísticas generales
    print("📊 ESTADÍSTICAS:")
    print("-" * 30)
    print(f"   • Tipos únicos: {len(tipos_counter)}")
    print(f"   • Subtipos únicos: {len(subtipos_counter)}")
    print(f"   • Combinaciones únicas: {len(combinaciones)}")
    
    # Mostrar entidades sin tipo o subtipo
    sin_tipo = sum(1 for e in entidades if not e.get('entity_type'))
    sin_subtipo = sum(1 for e in entidades if not e.get('subtype'))
    
    if sin_tipo > 0:
        print(f"   • Entidades sin tipo: {sin_tipo}")
    if sin_subtipo > 0:
        print(f"   • Entidades sin subtipo: {sin_subtipo}")
    
    return {
        'tipos': dict(tipos_counter),
        'subtipos': dict(subtipos_counter),
        'combinaciones': dict(combinaciones),
        'total_entidades': len(entidades)
    }

# Ejecutar análisis
input_file = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl"

# Verificar que existe el archivo
input_path = Path(input_file)
if not input_path.exists():
    print(f"❌ ERROR: No existe el archivo: {input_file}")
else:
    print(f"✅ Archivo encontrado: {input_path.name}")
    stats = analizar_tipos_entidades(input_file)
    
    print(f"\n🎯 RESUMEN:")
    print(f"   • Total de entidades: {stats['total_entidades']}")
    print(f"   • Tipos principales: {list(stats['tipos'].keys())[:5]}")
    print(f"   • Subtipos principales: {list(stats['subtipos'].keys())[:5]}")

✅ Archivo encontrado: enriched_data_5.jsonl
�� ANALIZANDO TIPOS Y SUBTIPOS DE ENTIDADES
📥 Archivo: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl

 Entidades cargadas: 124

��️  TIPOS DE ENTIDADES:
----------------------------------------
   • site: 124

📋 SUBTIPOS DE ENTIDADES:
----------------------------------------
   • island: 22
   • city: 20
   • airport: 18
   • temple: 13
   • province: 4
   • national park: 4
   • town: 4
   • Temple: 3
   • Historical Park: 2
   • beach: 2
   • Marine Park: 2
   • historical site: 2
   • City: 2
   • mall: 2
   • archipielago: 1
   • mountain: 1
   • waterfall: 1
   • sanctuary: 1
   • bay: 1
   • peninsula: 1
   • floating market: 1
   • landmark: 1
   • country: 1
   • historical complex: 1
   • archaeological park: 1
   • Island: 1
   • train station: 1
   • Town: 1
   • Province: 1
   • Park: 1
   • Waterfall: 1
   • Garden: 1
   • cave: 1
   • museum: 1
   • street: 1
   • market: 1
   • natural park: 1
   • 

In [37]:
import json
from pathlib import Path

def normalizar_tipos_subtipos(input_path, output_path):
    """Normaliza tipos y subtipos según las reglas especificadas"""
    
    print("�� NORMALIZANDO TIPOS Y SUBTIPOS")
    print("=" * 50)
    print(f"📥 Entrada: {input_path}")
    print(f"📤 Salida: {output_path}")
    print()
    
    # Cargar entidades
    entidades = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                entidades.append(json.loads(line))
    
    print(f" Entidades cargadas: {len(entidades)}")
    
    # Contadores para cambios
    cambios_tipo = 0
    cambios_subtipo = 0
    
    # Procesar cada entidad
    for entidad in entidades:
        # Normalizar tipo a minúsculas
        tipo_original = entidad.get('entity_type', '')
        if tipo_original:
            tipo_nuevo = tipo_original.lower()
            if tipo_nuevo != tipo_original:
                entidad['entity_type'] = tipo_nuevo
                cambios_tipo += 1
        
        # Normalizar subtipo a minúsculas y aplicar reglas
        subtipo_original = entidad.get('subtype', '')
        if subtipo_original:
            subtipo_nuevo = subtipo_original.lower()
            
            # Aplicar reglas de normalización
            if subtipo_nuevo in ['historical park', 'historical site', 'historical complex', 'archaeological park']:
                subtipo_nuevo = 'historic'
                print(f"✅ {entidad.get('name', 'N/A')}: {subtipo_original} → historic")
            
            elif subtipo_nuevo in ['national park', 'marine park']:
                subtipo_nuevo = 'natural park'
                print(f"✅ {entidad.get('name', 'N/A')}: {subtipo_original} → natural park")
            
            elif subtipo_nuevo in ['town', 'village']:
                subtipo_nuevo = 'city'
                print(f"✅ {entidad.get('name', 'N/A')}: {subtipo_original} → city")
            
            # Aplicar cambio si es diferente
            if subtipo_nuevo != subtipo_original:
                entidad['subtype'] = subtipo_nuevo
                cambios_subtipo += 1
    
    # Guardar archivo normalizado
    with open(output_path, 'w', encoding='utf-8') as f:
        for entidad in entidades:
            f.write(json.dumps(entidad, ensure_ascii=False) + '\n')
    
    print(f"\n�� RESUMEN DE CAMBIOS:")
    print("-" * 30)
    print(f"   • Tipos normalizados: {cambios_tipo}")
    print(f"   • Subtipos normalizados: {cambios_subtipo}")
    print(f"   • Total de cambios: {cambios_tipo + cambios_subtipo}")
    print(f"   • Archivo guardado: {output_path}")
    
    # Mostrar estadísticas finales
    print(f"\n📈 ESTADÍSTICAS FINALES:")
    print("-" * 30)
    
    tipos_finales = {}
    subtipos_finales = {}
    
    for entidad in entidades:
        tipo = entidad.get('entity_type', 'sin_tipo')
        subtipo = entidad.get('subtype', 'sin_subtipo')
        
        tipos_finales[tipo] = tipos_finales.get(tipo, 0) + 1
        subtipos_finales[subtipo] = subtipos_finales.get(subtipo, 0) + 1
    
    print("Tipos finales:")
    for tipo, count in sorted(tipos_finales.items()):
        print(f"   • {tipo}: {count}")
    
    print("\nSubtipos finales:")
    for subtipo, count in sorted(subtipos_finales.items()):
        print(f"   • {subtipo}: {count}")
    
    return {
        'cambios_tipo': cambios_tipo,
        'cambios_subtipo': cambios_subtipo,
        'total_cambios': cambios_tipo + cambios_subtipo,
        'tipos_finales': tipos_finales,
        'subtipos_finales': subtipos_finales
    }

# Ejecutar normalización
input_file = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl"
output_file = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl"

# Verificar que existe el archivo de entrada
input_path = Path(input_file)
if not input_path.exists():
    print(f"❌ ERROR: No existe el archivo: {input_file}")
else:
    print(f"✅ Archivo encontrado: {input_path.name}")
    stats = normalizar_tipos_subtipos(input_file, output_file)
    
    print(f"\n🎉 ¡NORMALIZACIÓN COMPLETADA!")
    print(f"📁 Archivo normalizado: {output_file}")

✅ Archivo encontrado: enriched_data_5.jsonl
�� NORMALIZANDO TIPOS Y SUBTIPOS
📥 Entrada: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl
📤 Salida: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl

 Entidades cargadas: 124

�� RESUMEN DE CAMBIOS:
------------------------------
   • Tipos normalizados: 0
   • Subtipos normalizados: 0
   • Total de cambios: 0
   • Archivo guardado: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl

📈 ESTADÍSTICAS FINALES:
------------------------------
Tipos finales:
   • site: 124

Subtipos finales:
   • airport: 18
   • archipielago: 1
   • bay: 1
   • beach: 2
   • cave: 1
   • city: 28
   • country: 1
   • floating market: 1
   • garden: 1
   • historic: 6
   • island: 23
   • landmark: 1
   • mall: 2
   • market: 1
   • mountain: 1
   • museum: 1
   • natural park: 7
   • park: 1
   • peninsula: 1
   • province: 5
   • sanctuary: 1
   • street: 1
   • temple: 16
   • train s

## We print the entities without subtype

In [33]:
import json
from pathlib import Path

def mostrar_entidades_sin_tipo(input_path):
    """Muestra las entidades sin tipo con su posición en el archivo"""
    
    print(" ENTIDADES SIN TIPO")
    print("=" * 50)
    print(f"📥 Archivo: {input_path}")
    print()
    
    # Cargar entidades
    entidades = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                entidades.append(json.loads(line))
    
    print(f" Entidades totales: {len(entidades)}")
    print()
    
    # Encontrar entidades sin tipo
    entidades_sin_tipo = []
    
    for i, entidad in enumerate(entidades, 1):
        tipo = entidad.get('entity_type', '').strip()
        if not tipo:
            entidades_sin_tipo.append((i, entidad))
    
    print(f" Entidades sin tipo encontradas: {len(entidades_sin_tipo)}")
    print()
    
    if entidades_sin_tipo:
        print("�� LISTADO DE ENTIDADES SIN TIPO:")
        print("-" * 50)
        
        for pos, entidad in entidades_sin_tipo:
            nombre = entidad.get('name', 'Sin nombre')
            subtipo = entidad.get('subtype', 'Sin subtipo')
            descripcion = entidad.get('description', 'Sin descripción')
            
            # Truncar descripción si es muy larga
            if len(descripcion) > 100:
                descripcion = descripcion[:100] + "..."
            
            print(f"{pos:3d}/{len(entidades)}. {nombre}")
            print(f"     • Subtipo: {subtipo}")
            print(f"     • Descripción: {descripcion}")
            
            # Mostrar otros campos relevantes
            appearances = entidad.get('appearances', 0)
            if appearances > 0:
                print(f"     • Apariciones: {appearances}")
            
            # Mostrar si tiene datos de Nominatim
            nominatim_data = entidad.get('nominatim_match', {})
            if nominatim_data.get('found'):
                quality = nominatim_data.get('quality_level', 'N/A')
                print(f"     • Nominatim: {quality}")
            
            print()
    else:
        print("✅ No se encontraron entidades sin tipo")
    
    # Estadísticas
    porcentaje = (len(entidades_sin_tipo) / len(entidades)) * 100 if entidades else 0
    print(f"📊 ESTADÍSTICAS:")
    print(f"   • Entidades sin tipo: {len(entidades_sin_tipo)}")
    print(f"   • Porcentaje: {porcentaje:.1f}%")
    print(f"   • Posiciones: {[pos for pos, _ in entidades_sin_tipo] if entidades_sin_tipo else 'Ninguna'}")

# Ejecutar análisis
input_file = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl"

# Verificar que existe el archivo
input_path = Path(input_file)
if not input_path.exists():import json
from pathlib import Path

def mostrar_entidades_incompletas(input_path):
    """Muestra las entidades sin tipo o sin subtipo con su posición"""
    
    print(" ENTIDADES SIN TIPO O SIN SUBTIPO")
    print("=" * 60)
    print(f"📥 Archivo: {input_path}")
    print()
    
    # Cargar entidades
    entidades = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                entidades.append(json.loads(line))
    
    print(f" Entidades totales: {len(entidades)}")
    print()
    
    # Encontrar entidades incompletas
    entidades_sin_tipo = []
    entidades_sin_subtipo = []
    
    for i, entidad in enumerate(entidades, 1):
        tipo = entidad.get('entity_type', '').strip()
        subtipo = entidad.get('subtype', '').strip()
        
        if not tipo:
            entidades_sin_tipo.append((i, entidad))
        
        if not subtipo:
            entidades_sin_subtipo.append((i, entidad))
    
    # Mostrar entidades sin tipo
    print(f" Entidades sin tipo: {len(entidades_sin_tipo)}")
    print(f" Entidades sin subtipo: {len(entidades_sin_subtipo)}")
    print()
    
    if entidades_sin_tipo:
        print(" ENTIDADES SIN TIPO:")
        print("-" * 50)
        
        for pos, entidad in entidades_sin_tipo:
            nombre = entidad.get('name', 'Sin nombre')
            subtipo = entidad.get('subtype', 'Sin subtipo')
            descripcion = entidad.get('description', 'Sin descripción')
            
            # Truncar descripción si es muy larga
            if len(descripcion) > 100:
                descripcion = descripcion[:100] + "..."
            
            print(f"{pos:3d}/{len(entidades)}. {nombre}")
            print(f"     • Subtipo: {subtipo}")
            print(f"     • Descripción: {descripcion}")
            
            # Mostrar otros campos relevantes
            appearances = entidad.get('appearances', 0)
            if appearances > 0:
                print(f"     • Apariciones: {appearances}")
            
            print()
    
    # Mostrar entidades sin subtipo
    if entidades_sin_subtipo:
        print(" ENTIDADES SIN SUBTIPO:")
        print("-" * 50)
        
        for pos, entidad in entidades_sin_subtipo:
            nombre = entidad.get('name', 'Sin nombre')
            tipo = entidad.get('entity_type', 'Sin tipo')
            descripcion = entidad.get('description', 'Sin descripción')
            
            # Truncar descripción si es muy larga
            if len(descripcion) > 100:
                descripcion = descripcion[:100] + "..."
            
            print(f"{pos:3d}/{len(entidades)}. {nombre}")
            print(f"     • Tipo: {tipo}")
            print(f"     • Descripción: {descripcion}")
            
            # Mostrar otros campos relevantes
            appearances = entidad.get('appearances', 0)
            if appearances > 0:
                print(f"     • Apariciones: {appearances}")
            
            print()
    
    # Si no hay entidades incompletas
    if not entidades_sin_tipo and not entidades_sin_subtipo:
        print("✅ No se encontraron entidades sin tipo o sin subtipo")
    
    # Estadísticas
    porcentaje_sin_tipo = (len(entidades_sin_tipo) / len(entidades)) * 100 if entidades else 0
    porcentaje_sin_subtipo = (len(entidades_sin_subtipo) / len(entidades)) * 100 if entidades else 0
    
    print(f"📊 ESTADÍSTICAS:")
    print(f"   • Entidades sin tipo: {len(entidades_sin_tipo)} ({porcentaje_sin_tipo:.1f}%)")
    print(f"   • Entidades sin subtipo: {len(entidades_sin_subtipo)} ({porcentaje_sin_subtipo:.1f}%)")
    
    if entidades_sin_tipo:
        posiciones_sin_tipo = [pos for pos, _ in entidades_sin_tipo]
        print(f"   • Posiciones sin tipo: {posiciones_sin_tipo}")
    
    if entidades_sin_subtipo:
        posiciones_sin_subtipo = [pos for pos, _ in entidades_sin_subtipo]
        print(f"   • Posiciones sin subtipo: {posiciones_sin_subtipo}")

# Ejecutar análisis
input_file = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl"

# Verificar que existe el archivo
input_path = Path(input_file)
if not input_path.exists():
    print(f"❌ ERROR: No existe el archivo: {input_file}")
else:
    print(f"✅ Archivo encontrado: {input_path.name}")
    mostrar_entidades_incompletas(input_file)
    print(f"❌ ERROR: No existe el archivo: {input_file}")


✅ Archivo encontrado: enriched_data_5.jsonl
 ENTIDADES SIN TIPO O SIN SUBTIPO
📥 Archivo: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl

 Entidades totales: 124

 Entidades sin tipo: 0
 Entidades sin subtipo: 0

✅ No se encontraron entidades sin tipo o sin subtipo
📊 ESTADÍSTICAS:
   • Entidades sin tipo: 0 (0.0%)
   • Entidades sin subtipo: 0 (0.0%)
❌ ERROR: No existe el archivo: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl


## We are goint to properly structure the jerarquy

### We add Thailand to all the elements

In [42]:
import json
from pathlib import Path

def anadir_jerarquia_thailand(input_path, output_path):
    """Añade jerarquía de Tailandia a todas las entidades excepto al propio país"""
    
    print("🔄 AÑADIENDO JERARQUÍA DE TAILANDIA")
    print("=" * 50)
    print(f"📥 Entrada: {input_path}")
    print(f"📤 Salida: {output_path}")
    print()
    
    # Cargar entidades
    entidades = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                entidades.append(json.loads(line))
    
    print(f" Entidades cargadas: {len(entidades)}")
    
    # Procesar cada entidad
    entidades_actualizadas = []
    cambios_realizados = 0
    normalizaciones = 0
    
    for entidad in entidades:
        nombre = entidad.get('name', '').strip()
        subtipo = entidad.get('subtype', '').strip()
        
        # No añadir jerarquía al propio país Tailandia
        if nombre.lower() == 'thailand' and subtipo.lower() == 'country':
            entidades_actualizadas.append(entidad)
            continue
        
        # Verificar si ya tiene jerarquía
        hierarchy_actual = entidad.get('hierarchy', [])
        
        # Normalizar "Tailandia" a "Thailand" en la jerarquía existente
        hierarchy_normalizada = []
        for item in hierarchy_actual:
            if (item.get('type') == 'country' and 
                item.get('name', '').lower() in ['thailand', 'tailandia']):
                # Normalizar a "Thailand"
                item_normalizado = item.copy()
                item_normalizado['name'] = 'Thailand'
                item_normalizado['code'] = 'TH'
                hierarchy_normalizada.append(item_normalizado)
                if item.get('name', '').lower() == 'tailandia':
                    normalizaciones += 1
                    print(f"🔄 {nombre}: Normalizado 'Tailandia' → 'Thailand'")
            else:
                hierarchy_normalizada.append(item)
        
        # Buscar si ya tiene Tailandia en la jerarquía normalizada
        tiene_thailand = any(
            item.get('type') == 'country' and 
            item.get('name', '').lower() == 'thailand'
            for item in hierarchy_normalizada
        )
        
        if not tiene_thailand:
            # Añadir Tailandia al principio de la jerarquía
            nueva_hierarchy = [
                {"type": "country", "name": "Thailand", "code": "TH"}
            ] + hierarchy_normalizada
            
            entidad['hierarchy'] = nueva_hierarchy
            cambios_realizados += 1
            print(f"✅ {nombre}: Añadida jerarquía de Thailand")
        else:
            # Usar la jerarquía normalizada
            entidad['hierarchy'] = hierarchy_normalizada
        
        entidades_actualizadas.append(entidad)
    
    # Guardar archivo actualizado
    with open(output_path, 'w', encoding='utf-8') as f:
        for entidad in entidades_actualizadas:
            f.write(json.dumps(entidad, ensure_ascii=False) + '\n')
    
    print(f"\n📊 RESUMEN:")
    print(f"   • Entidades procesadas: {len(entidades)}")
    print(f"   • Jerarquías añadidas: {cambios_realizados}")
    print(f"   • Normalizaciones 'Tailandia' → 'Thailand': {normalizaciones}")
    print(f"   • Archivo guardado: {output_path}")

# Ejecutar
input_file = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl"
output_file = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_6.jsonl"

input_path = Path(input_file)
if not input_path.exists():
    print(f"❌ ERROR: No existe el archivo: {input_file}")
else:
    print(f"✅ Archivo encontrado: {input_path.name}")
    anadir_jerarquia_thailand(input_file, output_file)
    print(f"\n�� ¡Jerarquía añadida y normalizada!")

✅ Archivo encontrado: enriched_data_5.jsonl
🔄 AÑADIENDO JERARQUÍA DE TAILANDIA
📥 Entrada: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_5.jsonl
📤 Salida: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_6.jsonl

 Entidades cargadas: 124
🔄 Ayutthaya: Normalizado 'Tailandia' → 'Thailand'
✅ Koh Samui: Añadida jerarquía de Thailand
✅ Phuket: Añadida jerarquía de Thailand
✅ Sukhothai: Añadida jerarquía de Thailand
✅ Koh Lipe: Añadida jerarquía de Thailand
🔄 Chiang Mai: Normalizado 'Tailandia' → 'Thailand'
✅ Koh Phangan: Añadida jerarquía de Thailand
🔄 Pai: Normalizado 'Tailandia' → 'Thailand'
🔄 Kanchannaburi: Normalizado 'Tailandia' → 'Thailand'
✅ Doi Inthanon: Añadida jerarquía de Thailand
🔄 Kanchanaburi: Normalizado 'Tailandia' → 'Thailand'
✅ Phitsanulok: Añadida jerarquía de Thailand
✅ Similan Islands: Añadida jerarquía de Thailand
✅ Sukhothai Historical Park: Añadida jerarquía de Thailand
✅ Wat Rong Khun: Añadida jerarquía de Thailand
🔄 Koh Samet: 

###  Añadimos manualmente las otras jerarquias

In [9]:
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import json
from pathlib import Path
from collections import defaultdict

class HierarchyEditor:
    def __init__(self, root, input_path, output_path):
        self.root = root
        self.root.title("Editor de Jerarquías - TravelApp")
        self.root.geometry("1600x1000")
        
        # Rutas de archivos
        self.input_path = Path(input_path)
        self.output_path = Path(output_path)
        
        # Cargar datos
        self.entidades = self.cargar_entidades()
        self.indice_actual = 0
        
        # Subtipos válidos para jerarquía
        self.subtipos_validos = ['city', 'province', 'island', 'archipielago', 'peninsula', 'town', 'bay', 'country']
        
        # Crear diccionario de entidades por subtipo
        self.entidades_por_subtipo = self.organizar_por_subtipo()
        
        self.crear_interfaz()
        self.mostrar_entidad_actual()

    def crear_interfaz(self):
        """Crea la interfaz gráfica"""
        # Frame principal
        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

        # Configurar grid
        self.root.columnconfigure(0, weight=1)
        self.root.rowconfigure(0, weight=1)
        main_frame.columnconfigure(1, weight=1)
        main_frame.rowconfigure(2, weight=1)

        # Información de archivo
        ttk.Label(main_frame, text=f"Archivo: {self.input_path.name}", font=("Arial", 12, "bold")).grid(row=0, column=0, columnspan=4, sticky=tk.W, pady=(0, 10))

        # Contador de entidades - MÁS GRANDE Y VISIBLE
        self.contador_label = ttk.Label(main_frame, text="", font=("Arial", 14, "bold"), foreground="blue")
        self.contador_label.grid(row=1, column=0, columnspan=4, sticky=tk.W, pady=(0, 15))

        # Frame izquierdo - Navegación y entidad actual
        left_frame = ttk.Frame(main_frame)
        left_frame.grid(row=2, column=0, sticky=(tk.W, tk.E, tk.N, tk.S), padx=(0, 10))

        # Botones de navegación
        ttk.Button(left_frame, text="⏮️ Primera", command=self.primera_entidad, width=15).pack(fill=tk.X, pady=2)
        ttk.Button(left_frame, text="⏪ Anterior", command=self.anterior_entidad, width=15).pack(fill=tk.X, pady=2)
        ttk.Button(left_frame, text="⏩ Siguiente", command=self.siguiente_entidad, width=15).pack(fill=tk.X, pady=2)
        ttk.Button(left_frame, text="⏭️ Última", command=self.ultima_entidad, width=15).pack(fill=tk.X, pady=2)

        # Botones de acciones
        ttk.Button(left_frame, text="💾 Guardar", command=self.guardar_entidades, width=15).pack(fill=tk.X, pady=10)
        ttk.Button(left_frame, text="🔄 Recargar", command=self.recargar_entidades, width=15).pack(fill=tk.X, pady=2)
        ttk.Button(left_frame, text="➕ AÑADIR ESTA ENTIDAD", command=self.anadir_entidad_actual, width=15).pack(fill=tk.X, pady=10)

        # Frame central - Entidad actual y edición de jerarquía
        center_frame = ttk.Frame(main_frame)
        center_frame.grid(row=2, column=1, sticky=(tk.W, tk.E, tk.N, tk.S), padx=(0, 10))

        # Información de la entidad actual
        ttk.Label(center_frame, text="ENTIDAD ACTUAL:", font=("Arial", 12, "bold")).pack(anchor=tk.W)

        self.entidad_info = ttk.Label(center_frame, text="", font=("Arial", 10))
        self.entidad_info.pack(anchor=tk.W, pady=(0, 10))

        # Jerarquía actual
        ttk.Label(center_frame, text="JERARQUÍA ACTUAL:", font=("Arial", 12, "bold")).pack(anchor=tk.W)

        self.jerarquia_text = scrolledtext.ScrolledText(center_frame, height=8, width=50, font=("Consolas", 9))
        self.jerarquia_text.pack(fill=tk.BOTH, expand=True, pady=(0, 10))

        # FRAME PARA AÑADIR ELEMENTOS A LA JERARQUÍA
        hierarchy_edit_frame = ttk.LabelFrame(center_frame, text="AÑADIR A JERARQUÍA", padding="5")
        hierarchy_edit_frame.pack(fill=tk.X, pady=(0, 10))

        # Tipo de jerarquía
        ttk.Label(hierarchy_edit_frame, text="Tipo:").grid(row=0, column=0, sticky=tk.W, padx=(0, 5))
        self.tipo_var = tk.StringVar(value="city")
        tipo_combo = ttk.Combobox(hierarchy_edit_frame, textvariable=self.tipo_var, values=self.subtipos_validos, width=15)
        tipo_combo.grid(row=0, column=1, sticky=tk.W, padx=(0, 10))

        # Nombre de jerarquía (combobox dependiente del subtipo)
        ttk.Label(hierarchy_edit_frame, text="Entidad:").grid(row=0, column=2, sticky=tk.W, padx=(0, 5))
        self.nombre_var = tk.StringVar()
        self.nombre_combo = ttk.Combobox(hierarchy_edit_frame, textvariable=self.nombre_var, width=30)
        self.nombre_combo.grid(row=0, column=3, sticky=tk.W, padx=(0, 10))

        def actualizar_nombres_entidades(*args):
            subtipo = self.tipo_var.get()
            nombres_entidades = [ent.get('name', 'Sin nombre') for ent in self.entidades if ent.get('subtype') == subtipo]
            self.nombre_combo['values'] = nombres_entidades
            if nombres_entidades:
                self.nombre_var.set(nombres_entidades[0])
            else:
                self.nombre_var.set("")

        # Actualiza la lista de entidades cuando cambias el tipo
        self.tipo_var.trace('w', actualizar_nombres_entidades)
        actualizar_nombres_entidades()

        # Código (opcional)
        ttk.Label(hierarchy_edit_frame, text="Código:").grid(row=0, column=4, sticky=tk.W, padx=(0, 5))
        self.codigo_var = tk.StringVar()
        codigo_entry = ttk.Entry(hierarchy_edit_frame, textvariable=self.codigo_var, width=8)
        codigo_entry.grid(row=0, column=5, sticky=tk.W, padx=(0, 10))

        # Botón para añadir
        ttk.Button(hierarchy_edit_frame, text="➕ Añadir", command=self.anadir_a_jerarquia).grid(row=0, column=6, padx=(0, 5))

        # Botón para eliminar último elemento
        ttk.Button(hierarchy_edit_frame, text="🗑️ Eliminar último", command=self.eliminar_ultimo_jerarquia).grid(row=0, column=7)

        # Frame derecho - Lista de entidades por subtipo
        right_frame = ttk.Frame(main_frame)
        right_frame.grid(row=2, column=2, sticky=(tk.W, tk.E, tk.N, tk.S), padx=(0, 10))

        ttk.Label(right_frame, text="ENTIDADES POR SUBTIPO:", font=("Arial", 12, "bold")).pack(anchor=tk.W)

        # Notebook para pestañas de subtipos
        self.notebook = ttk.Notebook(right_frame)
        self.notebook.pack(fill=tk.BOTH, expand=True)

        self.crear_pestanas_subtipos()

        # Frame inferior - Área de visualización completa
        bottom_frame = ttk.Frame(main_frame)
        bottom_frame.grid(row=3, column=0, columnspan=4, sticky=(tk.W, tk.E, tk.N, tk.S), pady=(10, 0))

        ttk.Label(bottom_frame, text="JSON COMPLETO:", font=("Arial", 12, "bold")).pack(anchor=tk.W)
        self.text_area = scrolledtext.ScrolledText(bottom_frame, height=12, width=120, font=("Consolas", 9))
        self.text_area.pack(fill=tk.BOTH, expand=True)    
        
    def cargar_entidades(self):
        """Carga las entidades desde el archivo"""
        entidades = []
        try:
            with open(self.input_path, 'r', encoding='utf-8') as f:
                for line in f:
                    if line.strip():
                        entidades.append(json.loads(line))
            print(f"✅ Cargadas {len(entidades)} entidades desde {self.input_path}")
            return entidades
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo cargar el archivo: {e}")
            return []
    
    def organizar_por_subtipo(self):
        """Organiza las entidades por subtipo para la interfaz"""
        organizadas = defaultdict(list)
        
        for entidad in self.entidades:
            subtipo = entidad.get('subtype', '').strip()
            if subtipo in self.subtipos_validos:
                organizadas[subtipo].append(entidad)
        
        return dict(organizadas)
    
    def guardar_entidades(self):
        """Guarda las entidades en el archivo de salida"""
        try:
            with open(self.output_path, 'w', encoding='utf-8') as f:
                for entidad in self.entidades:
                    f.write(json.dumps(entidad, ensure_ascii=False) + '\n')
            print(f"✅ Entidades guardadas en: {self.output_path}")
            messagebox.showinfo("Éxito", f"Entidades guardadas en:\n{self.output_path}")
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo guardar: {e}")
    
    
    
    def anadir_entidad_actual(self):
        """Añade la entidad actual a la jerarquía de otra entidad"""
        if not self.entidades:
            return
        
        entidad_actual = self.entidades[self.indice_actual]
        nombre_actual = entidad_actual.get('name', 'Sin nombre')
        subtipo_actual = entidad_actual.get('subtype', 'Sin subtipo')
        
        # Crear ventana de selección
        self.crear_ventana_seleccion_entidad(nombre_actual, subtipo_actual)
    
    def crear_ventana_seleccion_entidad(self, nombre_actual, subtipo_actual):
        """Crea una ventana para seleccionar a qué entidad añadir la jerarquía"""
        ventana = tk.Toplevel(self.root)
        ventana.title(f"Añadir '{nombre_actual}' a jerarquía")
        ventana.geometry("600x400")
        ventana.transient(self.root)
        ventana.grab_set()
        
        # Frame principal
        main_frame = ttk.Frame(ventana, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Título
        ttk.Label(main_frame, text=f"¿A qué entidad quieres añadir '{nombre_actual}' ({subtipo_actual})?", 
                 font=("Arial", 12, "bold")).pack(pady=(0, 10))
        
        # Frame para búsqueda
        search_frame = ttk.Frame(main_frame)
        search_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Label(search_frame, text="Buscar:").pack(side=tk.LEFT)
        search_var = tk.StringVar()
        search_entry = ttk.Entry(search_frame, textvariable=search_var, width=30)
        search_entry.pack(side=tk.LEFT, padx=(5, 0))
        
        # Lista de entidades
        listbox_frame = ttk.Frame(main_frame)
        listbox_frame.pack(fill=tk.BOTH, expand=True)
        
        listbox = tk.Listbox(listbox_frame, font=("Arial", 10))
        listbox.pack(fill=tk.BOTH, expand=True)
        
        # Función para filtrar entidades
        def filtrar_entidades(*args):
            listbox.delete(0, tk.END)
            busqueda = search_var.get().lower()
            
            for i, entidad in enumerate(self.entidades):
                nombre = entidad.get('name', '').lower()
                if busqueda in nombre:
                    listbox.insert(tk.END, f"{entidad.get('name')} ({entidad.get('subtype', 'N/A')})")
        
        # Binding para búsqueda
        search_var.trace('w', filtrar_entidades)
        
        # Cargar todas las entidades inicialmente
        filtrar_entidades()
        
        # Botones
        button_frame = ttk.Frame(main_frame)
        button_frame.pack(fill=tk.X, pady=(10, 0))
        
        def seleccionar_entidad():
            selection = listbox.curselection()
            if selection:
                # Obtener el nombre de la entidad seleccionada
                texto_seleccionado = listbox.get(selection[0])
                nombre_destino = texto_seleccionado.split(' (')[0]
                
                # Encontrar la entidad destino
                for i, entidad in enumerate(self.entidades):
                    if entidad.get('name') == nombre_destino:
                        # Añadir la entidad actual a la jerarquía de la entidad destino
                        entidad_destino = self.entidades[i]
                        hierarchy = entidad_destino.get('hierarchy', [])
                        
                        nuevo_elemento = {
                            "type": subtipo_actual,
                            "name": nombre_actual
                        }
                        
                        hierarchy.append(nuevo_elemento)
                        entidad_destino['hierarchy'] = hierarchy
                        
                        print(f"✅ Añadido '{nombre_actual}' ({subtipo_actual}) a la jerarquía de '{nombre_destino}'")
                        messagebox.showinfo("Éxito", f"Añadido '{nombre_actual}' a la jerarquía de '{nombre_destino}'")
                        
                        # Actualizar la visualización si la entidad destino es la actual
                        if i == self.indice_actual:
                            self.mostrar_entidad_actual()
                        
                        ventana.destroy()
                        return
                
                messagebox.showerror("Error", f"No se encontró la entidad: {nombre_destino}")
        
        ttk.Button(button_frame, text="✅ Añadir", command=seleccionar_entidad).pack(side=tk.LEFT, padx=(0, 5))
        ttk.Button(button_frame, text="❌ Cancelar", command=ventana.destroy).pack(side=tk.LEFT)
        
        # Focus en la búsqueda
        search_entry.focus()
    
    def anadir_a_jerarquia(self):
        """Añade un elemento a la jerarquía de la entidad actual"""
        if not self.entidades:
            return
        
        tipo = self.tipo_var.get().strip()
        nombre = self.nombre_var.get().strip()
        codigo = self.codigo_var.get().strip()
        
        if not tipo or not nombre:
            messagebox.showwarning("Advertencia", "Debes especificar tipo y nombre")
            return
        
        # Crear nuevo elemento de jerarquía
        nuevo_elemento = {"type": tipo, "name": nombre}
        if codigo:
            nuevo_elemento["code"] = codigo
        
        # Añadir a la jerarquía actual
        entidad = self.entidades[self.indice_actual]
        hierarchy = entidad.get('hierarchy', [])
        hierarchy.append(nuevo_elemento)
        entidad['hierarchy'] = hierarchy
        
        # Limpiar campos
        self.nombre_var.set("")
        self.codigo_var.set("")
        
        # Actualizar visualización
        self.mostrar_entidad_actual()
        print(f"✅ Añadido a jerarquía: {tipo} - {nombre}")
    
    def eliminar_ultimo_jerarquia(self):
        """Elimina el último elemento de la jerarquía"""
        if not self.entidades:
            return
        
        entidad = self.entidades[self.indice_actual]
        hierarchy = entidad.get('hierarchy', [])
        
        if hierarchy:
            elemento_eliminado = hierarchy.pop()
            entidad['hierarchy'] = hierarchy
            self.mostrar_entidad_actual()
            print(f"🗑️ Eliminado de jerarquía: {elemento_eliminado.get('type')} - {elemento_eliminado.get('name')}")
        else:
            messagebox.showinfo("Info", "La jerarquía ya está vacía")
    
    def crear_pestanas_subtipos(self):
        """Crea pestañas para cada subtipo"""
        for subtipo in self.subtipos_validos:
            if subtipo in self.entidades_por_subtipo:
                # Crear frame para la pestaña
                frame = ttk.Frame(self.notebook)
                self.notebook.add(frame, text=subtipo.upper())
                
                # Lista de entidades para este subtipo
                listbox = tk.Listbox(frame, font=("Arial", 9))
                listbox.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
                
                # Añadir entidades a la lista
                for entidad in self.entidades_por_subtipo[subtipo]:
                    nombre = entidad.get('name', 'Sin nombre')
                    listbox.insert(tk.END, nombre)
                
                # Binding para seleccionar entidad
                listbox.bind('<Double-Button-1>', lambda e, lb=listbox, st=subtipo: self.seleccionar_entidad_desde_lista(lb, st))
    
    def seleccionar_entidad_desde_lista(self, listbox, subtipo):
        """Selecciona una entidad desde la lista de subtipos"""
        selection = listbox.curselection()
        if selection:
            nombre_seleccionado = listbox.get(selection[0])
            
            # Encontrar la entidad en el índice global
            for i, entidad in enumerate(self.entidades):
                if entidad.get('name') == nombre_seleccionado and entidad.get('subtype') == subtipo:
                    self.indice_actual = i
                    self.mostrar_entidad_actual()
                    break
    
    def mostrar_entidad_actual(self):
        """Muestra la entidad actual en la interfaz"""
        if not self.entidades:
            self.contador_label.config(text="No hay entidades")
            self.entidad_info.config(text="")
            self.jerarquia_text.delete(1.0, tk.END)
            self.text_area.delete(1.0, tk.END)
            return
        
        entidad = self.entidades[self.indice_actual]
        
        # Actualizar contador - MÁS VISIBLE
        self.contador_label.config(text=f"📋 ENTIDAD {self.indice_actual + 1} DE {len(self.entidades)}")
        
        # Información de la entidad
        nombre = entidad.get('name', 'Sin nombre')
        subtipo = entidad.get('subtype', 'Sin subtipo')
        tipo = entidad.get('entity_type', 'Sin tipo')
        
        info_text = f"Nombre: {nombre}\nTipo: {tipo}\nSubtipo: {subtipo}"
        self.entidad_info.config(text=info_text)
        
        # Mostrar jerarquía actual
        hierarchy = entidad.get('hierarchy', [])
        self.jerarquia_text.delete(1.0, tk.END)
        
        if hierarchy:
            for i, item in enumerate(hierarchy, 1):
                tipo_hier = item.get('type', 'N/A')
                nombre_hier = item.get('name', 'N/A')
                codigo = item.get('code', '')
                codigo_text = f" ({codigo})" if codigo else ""
                self.jerarquia_text.insert(tk.END, f"{i}. {tipo_hier}: {nombre_hier}{codigo_text}\n")
        else:
            self.jerarquia_text.insert(tk.END, "Sin jerarquía")
        
        # Mostrar JSON completo
        json_str = json.dumps(entidad, indent=2, ensure_ascii=False)
        self.text_area.delete(1.0, tk.END)
        self.text_area.insert(1.0, json_str)
    
    def primera_entidad(self):
        """Va a la primera entidad"""
        if self.entidades:
            self.indice_actual = 0
            self.mostrar_entidad_actual()
    
    def anterior_entidad(self):
        """Va a la entidad anterior"""
        if self.entidades and self.indice_actual > 0:
            self.indice_actual -= 1
            self.mostrar_entidad_actual()
    
    def siguiente_entidad(self):
        """Va a la entidad siguiente"""
        if self.entidades and self.indice_actual < len(self.entidades) - 1:
            self.indice_actual += 1
            self.mostrar_entidad_actual()
    
    def ultima_entidad(self):
        """Va a la última entidad"""
        if self.entidades:
            self.indice_actual = len(self.entidades) - 1
            self.mostrar_entidad_actual()
    
    def recargar_entidades(self):
        """Recarga las entidades desde el archivo"""
        self.entidades = self.cargar_entidades()
        self.entidades_por_subtipo = self.organizar_por_subtipo()
        self.indice_actual = 0
        self.mostrar_entidad_actual()
        messagebox.showinfo("Éxito", "Entidades recargadas desde el archivo")

    

def iniciar_editor_hierarchy(input_path, output_path):
    """Función para iniciar el editor de jerarquías desde notebook"""
    root = tk.Tk()
    app = HierarchyEditor(root, input_path, output_path)
    root.mainloop()



In [10]:
# BLOQUE 2: EDITOR DE JERARQUÍAS
# ===============================

import sys
from pathlib import Path

# Añadir el directorio del script al path
script_dir = Path(r"D:\TravelApp\Project\scripts\scraper_enrichment\manual_postprocessing")
sys.path.append(str(script_dir))

# Importar el editor

# Configurar rutas
input_file = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_6.jsonl"
output_file = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_6_final.jsonl"

print("🚀 INICIANDO EDITOR DE JERARQUÍAS")
print("=" * 50)
print(f"�� Archivo de entrada: {input_file}")
print(f"�� Archivo de salida: {output_file}")
print()
print("📋 FUNCIONALIDADES:")
print("   • Navegación entre entidades")
print("   • Visualización por subtipos (city, province, island, etc.)")
print("   • Visualización de jerarquía actual")
print("   • AÑADIR elementos a la jerarquía")
print("   • ELIMINAR elementos de la jerarquía")
print("   • JSON completo de cada entidad")
print("   • Guardado de cambios")
print()
print("💡 INSTRUCCIONES:")
print("   1. Usa los botones de navegación para moverte entre entidades")
print("   2. Haz doble clic en las listas de subtipos para ir a una entidad específica")
print("   3. Revisa la jerarquía actual en el panel central")
print("   4. Para AÑADIR a jerarquía:")
print("      - Selecciona tipo (city, province, etc.)")
print("      - Escribe el nombre")
print("      - Opcionalmente añade código")
print("      - Clica '➕ Añadir'")
print("   5. Para ELIMINAR: clica '🗑️ Eliminar último'")
print("   6. Usa '💾 Guardar' para salvar los cambios")
print()

# Iniciar la interfaz
iniciar_editor_hierarchy(input_file, output_file)

🚀 INICIANDO EDITOR DE JERARQUÍAS
�� Archivo de entrada: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_6.jsonl
�� Archivo de salida: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_6_final.jsonl

📋 FUNCIONALIDADES:
   • Navegación entre entidades
   • Visualización por subtipos (city, province, island, etc.)
   • Visualización de jerarquía actual
   • AÑADIR elementos a la jerarquía
   • ELIMINAR elementos de la jerarquía
   • JSON completo de cada entidad
   • Guardado de cambios

💡 INSTRUCCIONES:
   1. Usa los botones de navegación para moverte entre entidades
   2. Haz doble clic en las listas de subtipos para ir a una entidad específica
   3. Revisa la jerarquía actual en el panel central
   4. Para AÑADIR a jerarquía:
      - Selecciona tipo (city, province, etc.)
      - Escribe el nombre
      - Opcionalmente añade código
      - Clica '➕ Añadir'
   5. Para ELIMINAR: clica '🗑️ Eliminar último'
   6. Usa '💾 Guardar' para salvar los cambios



## We plot how many sections are in the jsonl

In [41]:
import json
from collections import Counter

ruta = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_8.jsonl"

contador = Counter()
total = 0

with open(ruta, encoding="utf-8") as f:
    for line in f:
        if not line.strip():
            continue
        entidad = json.loads(line)
        for key in entidad.keys():
            contador[key] += 1
        total += 1

print(f"Total de entidades: {total}\n")
print("Claves (secciones) encontradas, ordenadas por ocurrencias:\n")
for key, count in contador.most_common():
    print(f"{key}: {count}")

Total de entidades: 124

Claves (secciones) encontradas, ordenadas por ocurrencias:

entity_type: 124
name: 124
subtype: 124
hierarchy: 124
normalized_name: 124
appearances: 124
all_source_urls: 124
lat: 102
lon: 102
type: 102
description: 96
wikidata_id: 32
avg_visit_duration: 28


In [ ]:
import sys
import os
from pathlib import Path

# Obtener la ruta absoluta del archivo
current_dir = Path.cwd()  # manual_postprocessing
auxs_dir = current_dir.parent / 'auxs'  # subir un nivel y entrar en auxs
normalized_cities_path = auxs_dir / 'normalized_cities.py'

# Importar usando importlib
import importlib.util
spec = importlib.util.spec_from_file_location("normalized_cities", str(normalized_cities_path))
normalized_cities = importlib.util.module_from_spec(spec)
sys.modules["normalized_cities"] = normalized_cities
spec.loader.exec_module(normalized_cities)

# Extraer las funciones y variables
NORMALIZED_CITIES = normalized_cities.NORMALIZED_CITIES
normalize_city = normalized_cities.normalize_city
get_city_info = normalized_cities.get_city_info

# Probar la normalización
print("=== PRUEBA DE NORMALIZACIÓN DE CIUDADES ===")

# Lista de ciudades para probar
ciudades_prueba = [
    "Bangkok City",
    "Chiangmai", 
    "Ko Samui",
    "Phi Phi",
    "Krung Thep",
    "Ciudad Desconocida"
]

for ciudad in ciudades_prueba:
    normalizada = normalize_city(ciudad)
    info = get_city_info(ciudad)
    
    if info:
        print(f"✅ {ciudad} → {normalizada}")
        print(f"   Variaciones: {info['variations']}")
    else:
        print(f"❌ {ciudad} → {normalizada} (no normalizada)")

print(f"\n=== ESTADÍSTICAS ===")
print(f"Total de ciudades normalizadas: {len(NORMALIZED_CITIES)}")

# Mostrar todas las ciudades con sus variaciones
print(f"\n=== TODAS LAS CIUDADES CON VARIACIONES ===")
for ciudad, variaciones in sorted(NORMALIZED_CITIES.items()):
    print(f"🏙️  {ciudad}:")
    for var in variaciones:
        print(f"   • {var}")
    print()

=== PRUEBA DE NORMALIZACIÓN DE CIUDADES ===
✅ Bangkok City → Bangkok
   Variaciones: ['Bangkok', 'Bangkok City', 'Krung Thep', 'Krung Thep Maha Nakhon', 'BKK', 'กรุงเทพมหานคร']
✅ Chiangmai → Chiang Mai
   Variaciones: ['Chiang Mai', 'Chiangmai', 'Chiang-Mai', 'Chiang Mai City', 'เชียงใหม่']
✅ Ko Samui → Koh Samui
   Variaciones: ['Koh Samui', 'Ko Samui', 'Samui Island', 'Samui', 'เกาะสมุย']
✅ Phi Phi → Koh Phi Phi
   Variaciones: ['Koh Phi Phi', 'Ko Phi Phi', 'Phi Phi Islands', 'Phi Phi', 'เกาะพีพี']
✅ Krung Thep → Bangkok
   Variaciones: ['Bangkok', 'Bangkok City', 'Krung Thep', 'Krung Thep Maha Nakhon', 'BKK', 'กรุงเทพมหานคร']
❌ Ciudad Desconocida → Ciudad Desconocida (no normalizada)

=== ESTADÍSTICAS ===
Total de ciudades normalizadas: 15

=== TODAS LAS CIUDADES CON VARIACIONES ===
🏙️  Ayutthaya:
   • Ayutthaya
   • Ayutthaya Historical Park
   • Ayutthaya City
   • Phra Nakhon Si Ayutthaya
   • พระนครศรีอยุธยา

🏙️  Bangkok:
   • Bangkok
   • Bangkok City
   • Krung Thep
   • Krung

## We delete the innecesary sections 

In [17]:
import json
from pathlib import Path

input_path = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_6.jsonl"
output_path = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_7.jsonl"

# Claves a eliminar
claves_a_quitar = [
    "source_url", "location_text", "images", "user_impressions", "agrupado_por",
    "entidades_originales", "official_website", "official_websites", "osm_uid",
    "price", "restrictions", "wikidata", "currency", "security", "lat", "lon",
    "nearby_suggestions"
]

n_modificados = 0
n_total = 0

with open(input_path, encoding="utf-8") as fin, open(output_path, "w", encoding="utf-8") as fout:
    for line in fin:
        if not line.strip():
            continue
        entidad = json.loads(line)
        n_total += 1

        # 1. Meter source_url en all_source_urls y eliminar source_url
        source_url = entidad.get("source_url")
        if source_url:
            all_urls = entidad.get("all_source_urls", [])
            if source_url not in all_urls:
                all_urls.append(source_url)
                entidad["all_source_urls"] = all_urls
            n_modificados += 1
        if "source_url" in entidad:
            del entidad["source_url"]

        # 2-8. Eliminar las claves indicadas
        for clave in claves_a_quitar:
            if clave in entidad:
                del entidad[clave]

        # Guardar la entidad limpia
        fout.write(json.dumps(entidad, ensure_ascii=False) + "\n")

print(f"Guardado en: {output_path}")
print(f"Entidades procesadas: {n_total}")
print(f"Entidades en las que se añadió source_url a all_source_urls: {n_modificados}")

Guardado en: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_7.jsonl
Entidades procesadas: 124
Entidades en las que se añadió source_url a all_source_urls: 124


## Deletion of unnecesary info and adding lat, long and type to the entity with osm_id

In [44]:
import json
import requests
import time

input_file = "D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_7.jsonl"
output_file = "D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_8.jsonl"

def get_osm_info(osm_id):
    try:
        if isinstance(osm_id, str) and osm_id.startswith("OSM:"):
            osm_id = osm_id.replace("OSM:", "")
        
        # Determinar el tipo correctamente
        if osm_id.startswith("R"):
            osm_type = "R"
            osm_number = osm_id[1:]
        elif osm_id.startswith("W"):
            osm_type = "W"
            osm_number = osm_id[1:]
        elif osm_id.startswith("N"):  # ← AÑADIR ESTA LÍNEA
            osm_type = "N"
            osm_number = osm_id[1:]
        else:
            osm_type = "W"
            osm_number = osm_id

        url = f"https://nominatim.openstreetmap.org/lookup?osm_ids={osm_type}{osm_number}&format=json"
        print(f"  → URL: {url}")
        headers = {
            "User-Agent": "TravelAppDataScript/1.2 (4@dominio.com)"
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200 and response.json():
            data = response.json()[0]
            print(f"  → Respuesta API: {data}")
            lat = data.get('lat')
            lon = data.get('lon')
            typ = data.get('type')
            print(f"  → Extraído: lat={lat}, lon={lon}, type={typ}")
            return lat, lon, typ
        else:
            print(f"  → Error en respuesta: {response.status_code}")
            return None, None, None
    except Exception as e:
        print(f"  → Error: {e}")
        return None, None, None
with open(input_file, "r", encoding="utf-8") as fin, open(output_file, "w", encoding="utf-8") as fout:
    for line in fin:
        entity = json.loads(line)
        lat, lon, typ = None, None, None
        
        # 1. PRIORIDAD: Si tiene osm_id directo → LLAMAR A LA API
        if "osm_id" in entity and entity["osm_id"]:
            print(f"\n=== {entity['name']} ===")
            print(f"Tiene osm_id: {entity['osm_id']}")
            lat, lon, typ = get_osm_info(entity["osm_id"])
            time.sleep(1)  # Respetar rate limit
            
        # 2. Si NO tiene osm_id directo pero tiene nominatim_match → EXTRAER DE AHÍ
        elif "nominatim_match" in entity and entity["nominatim_match"] and entity["nominatim_match"].get("data"):
            print(f"\n=== {entity['name']} ===")
            print(f"Tiene nominatim_match")
            nom_data = entity["nominatim_match"]["data"]
            lat = nom_data.get("lat")
            lon = nom_data.get("lon")
            typ = nom_data.get("type")
            print(f"  → Extraído de nominatim: lat={lat}, lon={lon}, type={typ}")
        
        # 3. Mostrar lo que se va a añadir
        print(f"  → AÑADIENDO A LA ENTIDAD: lat={lat}, lon={lon}, type={typ}")
        
        # 4. Añadir los campos al nivel superior
        if lat is not None:
            entity["lat"] = lat
        if lon is not None:
            entity["lon"] = lon
        if typ is not None:
            entity["type"] = typ
            
        # 5. Eliminar los campos originales
        entity.pop("osm_id", None)
        entity.pop("nominatim_match", None)
        
        fout.write(json.dumps(entity, ensure_ascii=False) + "\n")

print("\nProceso completado!")


=== Bangkok ===
Tiene nominatim_match
  → Extraído de nominatim: lat=13.7524938, lon=100.4935089, type=administrative
  → AÑADIENDO A LA ENTIDAD: lat=13.7524938, lon=100.4935089, type=administrative

=== Ayutthaya ===
Tiene osm_id: OSM:R10839539
  → URL: https://nominatim.openstreetmap.org/lookup?osm_ids=R10839539&format=json
  → Respuesta API: {'place_id': 237853454, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 10839539, 'lat': '14.3535427', 'lon': '100.5645684', 'class': 'boundary', 'type': 'administrative', 'place_rank': 14, 'importance': 0.49306700633523093, 'addresstype': 'city', 'name': 'เทศบาลนครพระนครศรีอยุธยา', 'display_name': 'เทศบาลนครพระนครศรีอยุธยา, อำเภอพระนครศรีอยุธยา, จังหวัดพระนครศรีอยุธยา, 13000, ประเทศไทย', 'address': {'city': 'เทศบาลนครพระนครศรีอยุธยา', 'county': 'อำเภอพระนครศรีอยุธยา', 'province': 'จังหวัดพระนครศรีอยุธยา', 'ISO3166-2-lvl4': 'TH-14', 'postcode': '13000', 'country': 'ประเทศไทย'

In [45]:
import json

input_file = "D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_8.jsonl"

with open(input_file, "r", encoding="utf-8") as fin:
    for line in fin:
        entity = json.loads(line)
        if not entity.get("lat") or not entity.get("lon"):
            print(entity.get("name", "SIN NOMBRE"))

Thailand
Khao San Road
Hellfire Pass


## We plot the names and subtypes of the entities

In [52]:
import json

def print_entity_names_and_subtypes():
    # Lista para almacenar las entidades únicas
    entities = []
    
    # Leer el archivo JSONL
    with open('D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_9.jsonl', 'r', encoding='utf-8') as file:
        for line in file:
            try:
                data = json.loads(line.strip())
                
                # Extraer nombre y subtipo
                name = data.get('name', 'Sin nombre')
                subtype = data.get('subtype', 'Sin subtipo')
                
                # Crear tupla para evitar duplicados
                entity = (name, subtype)
                
                if entity not in entities:
                    entities.append(entity)
                    
            except json.JSONDecodeError:
                print(f"Error al parsear línea: {line}")
                continue
    
    # Ordenar por nombre
    entities.sort(key=lambda x: x[0])
    
    # Imprimir resultados
    print("ENTIDADES Y SUS SUBTIPOS:")
    print("=" * 50)
    
    for i, (name, subtype) in enumerate(entities, 1):
        print(f"{i:2d}. {name:<30} | {subtype}")
    
    print(f"\nTotal de entidades únicas: {len(entities)}")

# Ejecutar la función
if __name__ == "__main__":
    print_entity_names_and_subtypes()

ENTIDADES Y SUS SUBTIPOS:
 1. Ang Thong                      | natural park
 2. Ao Nang                        | city
 3. Ayutthaya                      | city
 4. Ayutthaya Historical Park      | historic
 5. Bahía de Phang Nga             | bay
 6. Bangkok                        | city
 7. Black Temple                   | museum
 8. Blue Temple                    | temple
 9. Buri Ram                       | city
10. Buri Ram Airport               | airport
11. Central World                  | mall
12. Chatuchak Market               | market
13. Chiang Khan                    | city
14. Chiang Mai                     | city
15. Chiang Mai International Airport | airport
16. Chiang Rai                     | city
17. Chiang Rai                     | province
18. Chiang Rai Airport             | airport
19. Chumphon                       | province
20. Chumphon                       | city
21. Damnoen Saduak Floating Market | floating market
22. Doi Inthanon                   | mountain

## We match some same entities

In [50]:
import json
from collections import defaultdict

def clean_and_merge_entities():
    # Diccionario para almacenar entidades fusionadas
    merged_entities = {}
    
    # Mapeo de nombres para fusionar
    name_mapping = {
        'Kanchannaburi': 'Kanchanaburi',
        'Ko Samui': 'Koh Samui', 
        'Ko Yao Noi': 'Koh Yao Noi'
    }
    
    # Leer el archivo JSONL de entrada
    input_file = r'D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_8.jsonl'
    output_file = r'D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_9.jsonl'
    
    with open(input_file, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                data = json.loads(line.strip())
                
                name = data.get('name', 'Sin nombre')
                subtype = data.get('subtype', 'Sin subtipo')
                
                # Saltar entidades a eliminar
                if name == 'Ayutthaya' and subtype == 'historic':
                    continue
                
                # Aplicar mapeo de nombres
                if name in name_mapping:
                    name = name_mapping[name]
                
                # Crear clave única
                key = (name, subtype)
                
                if key not in merged_entities:
                    merged_entities[key] = {
                        'entity_type': data.get('entity_type', 'site'),
                        'name': name,
                        'subtype': subtype,
                        'description': data.get('description', ''),
                        'hierarchy': data.get('hierarchy', []),
                        'avg_visit_duration': data.get('avg_visit_duration', ''),
                        'normalized_name': data.get('normalized_name', name),
                        'appearances': data.get('appearances', 0),
                        'all_source_urls': data.get('all_source_urls', []),
                        'lat': data.get('lat', ''),
                        'lon': data.get('lon', ''),
                        'type': data.get('type', ''),
                        'wikidata_id': data.get('wikidata_id', '')
                    }
                else:
                    # Sumar appearances y combinar URLs
                    merged_entities[key]['appearances'] += data.get('appearances', 0)
                    
                    # Combinar URLs únicas
                    existing_urls = set(merged_entities[key]['all_source_urls'])
                    new_urls = set(data.get('all_source_urls', []))
                    merged_entities[key]['all_source_urls'] = list(existing_urls.union(new_urls))
                    
                    # Combinar descripciones (mantener la más larga)
                    if len(data.get('description', '')) > len(merged_entities[key]['description']):
                        merged_entities[key]['description'] = data.get('description', '')
                    
            except json.JSONDecodeError:
                print(f"Error al parsear línea: {line}")
                continue
    
    # Ordenar por nombre
    sorted_entities = sorted(merged_entities.values(), key=lambda x: x['name'])
    
    # Escribir el archivo de salida
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for entity in sorted_entities:
            json.dump(entity, outfile, ensure_ascii=False)
            outfile.write('\n')
    
    # Imprimir resultados
    print("ENTIDADES LIMPIAS Y FUSIONADAS:")
    print("=" * 60)
    
    for i, entity in enumerate(sorted_entities, 1):
        print(f"{i:2d}. {entity['name']:<30} | {entity['subtype']:<15} | {entity['appearances']:2d} appearances")
    
    print(f"\nTotal de entidades únicas: {len(sorted_entities)}")
    print(f"Archivo guardado en: {output_file}")
    
    # Mostrar entidades fusionadas
    print("\nENTIDADES FUSIONADAS:")
    print("=" * 40)
    for original, mapped in name_mapping.items():
        print(f"'{original}' → '{mapped}'")
    
    return sorted_entities

# Ejecutar la función
if __name__ == "__main__":
    clean_and_merge_entities()

ENTIDADES LIMPIAS Y FUSIONADAS:
 1. Ang Thong                      | natural park    |  1 appearances
 2. Ao Nang                        | city            |  4 appearances
 3. Ayutthaya                      | city            | 26 appearances
 4. Ayutthaya Historical Park      | historic        |  1 appearances
 5. Bahía de Phang Nga             | bay             |  3 appearances
 6. Bangkok                        | city            | 32 appearances
 7. Black Temple                   | museum          |  1 appearances
 8. Blue Temple                    | temple          |  2 appearances
 9. Buri Ram                       | city            |  1 appearances
10. Buri Ram Airport               | airport         |  1 appearances
11. Central World                  | mall            |  1 appearances
12. Chatuchak Market               | market          |  1 appearances
13. Chiang Khan                    | city            |  1 appearances
14. Chiang Mai                     | city            | 12 